# Extracción de Información Estructurada de Resoluciones de Calificación Ambiental

Este notebook tiene como objetivo extraer de forma estructurada la información contenida en las Resoluciones de Calificación Ambiental. Para ello se definen funciones que permiten:
- Leer y limpiar tablas extraídas de archivos PDF.
- Fusionar tablas cuando corresponda.
- Detectar y asignar números o etiquetas a las tablas.
- Extraer secciones del texto mediante búsquedas difusas.

### Importar Librerías

In [ ]:
# ===============================
# Librerías Estándar de Python
# ===============================
import copy
import json
import os
import re
import shutil
import time
from time import sleep
import xml.etree.ElementTree as ET

# ===============================
# Librerías de Terceros
# ===============================
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import jsonschema
import numpy as np
import openpyxl
import pandas as pd
import pdfplumber
from PyPDF2 import PdfReader
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import tabula
from tabula.io import read_pdf
from transformers import AutoTokenizer

# ===============================
# Librerías de Entorno y AI
# ===============================
from dotenv import load_dotenv
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from openai import OpenAI
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

# ===============================
# Librerías para Pydantic
# ===============================
from enum import Enum
from pydantic import BaseModel, Field, constr, conlist, ValidationError
from typing import List, Optional, Literal, Annotated, Dict

### Establecer Directorios

In [ ]:
sector = "energia"
direccion_output = os.path.join(os.environ['USERPROFILE'], "RUTA", f"archivos_{sector}")
data_path = os.path.join(os.environ['USERPROFILE'], "RUTA", f"{sector}")

In [ ]:
load_dotenv()
api_key_openai = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(
    api_key=api_key_openai
)

### Definir Conceptos

In [ ]:
conceptos_dict = {
    "componente_materia": ["componente/materia", "componente ambiental", "componenete", "materia:", "componente ambiental o materia que regula la norma", "componente ambiental objeto de seguimiento", "materia regulada", "componente ambiental", "componente(s) ambiental(es) afectado(s)", "componente(s) ambiental(es) objeto de protección", "componente (s) ambiental (es) objeto de seguimiento"],
    "nombre_compromiso": ["nombre del compromiso", "compromiso ambiental voluntario"],
    "riesgo_contingencia": ["riesgo o contingencia", "riesgo/contingencia", "situación de riesgo o contingencia"],
    "normas": ["normas legales", "norma"],
    "fase": ["fase del proyecto", "fase del proyecto (fase de aplicación de la medida)", "fases", "fase", "fases de ejecución", "fase de ejecución", "fase de cumplimiento", "fase del proyecto a la que aplica o en la que se dará cumplimiento", "fase del proyecto", "fase del proyecto a la que aplica", "fase en que se presenta"],
    "parte_asociada": ["parte, obra o acción a la que aplica", "parte, obra, acción, emisión, residuo o sustancias a la que aplica", "parte, obra, acción, emisión, residuo o sustancias a la que aplica", "relación con el proyecto", "parte, obra o acción que lo genera", "parte, obra o acción asociada", "parte, obra, acción, emisión, residuo o sustancias a las que aplica", "parte u obra a la que aplica", "emplazamiento, parte, obra o acción asociada"],
    "lugar_forma_oportunidad": ["lugar, forma y oportunidad de implementación", "lugar de verificación", "forma y oportunidad de implementación"],
    "lugar": ["lugar de implementación", "lugar de implementación de la medida"], 
    "forma_oportunidad": ["forma y oportunidad de implementación", "forma y oportunidad de implementación de la medida"],
    "metodo_implementacion": ["método de implementación de la medida"],    
    "forma_cumplimiento": ["forma de cumplimiento"],
    "tipo_medida": ["tipo de medida", "medida(s) asociada(s)", "medida asociada"], 
    "indicador_cumplimiento": ["indicador que acredita su cumplimiento", "indicador de cumplimiento de la medida", "indicador de cumplimiento", "indicador que acredite su cumplimiento", "método o procedimiento de medición de cada parámetro"],
    "control_seguimiento": ["forma de control y seguimiento", "seguimiento y control"],
    "ice": ["referencia al ice para mayor detalle", "referencia al ice para mayores detalles", "referencia al ice", "referencia al ice o documentos del expediente de evaluación que contenga la descripción detallada", "referencia a documentos del expediente de evaluación que contenga la descripción detallada", "referencia al ice para mayores detalles sobre este impacto específico"],
    "otros_legal": ["otros cuerpos legales", "otros cuerpos legales asociados"],
    "impacto": ["impacto asociado", "impacto ambiental", "impactos asociados", "impacto ambiental asociado"],
    "objetivo_descripcion_justificacion": ["objetivo, descripción y justificación", "objetivo del compromiso", "descripción del compromiso", "justificación del compromiso"],
    "objetivo": ["objetivo", "objetivo de la medida"], 
    "descripcion": ["descripción", "descripción de la medida"],
    "justificacion": ["justificación", "justificación de la medida"],
    "calificacion": ["calificación de la parte u obra"],
    "condiciones": ["condiciones o exigencias específicas del pronunciamiento", "condiciones o exigencias específicas para su otorgamiento"],
    "pronunciamiento": ["pronunciamiento del órgano competente"],
    "acciones": ["acciones o medidas a implementar para prevenir la contingencia", "acciones a implementar", "acciones o medida a implementar para controlar la emergencia"],
    "sma": ["oportunidad y vías de comunicación a la sma de la activación del plan de emergencia", "oportunidad y vías de comunicación a la sma de la activación del plan", "oportunidad y vías de comunicación a la sma", "activación del plan de emergencia"],
    "mitigacion": ["mitigación", "nombre de la medida", "medida", "medida asociada", "medidas asociadas", "medida(s) asociada(s)"],
    "subcomponente": ["subcomponente", "subcomponente ambiental", "subcomponente ambiental objeto de seguimiento"],
    "medio_verificacion": ["medio de verificación"],
    "plazo_implementacion": ["plazo de implementación de la medida"],
    "indicador_exito": ["indicador de éxito", "indicador de efectividad de la medida"],
    "ubicacion": ["división político administrativa", "ubicación puntos de control", "ubicación de los puntos de control", "ubicación de los puntos o sitios de muestreo, medición, análisis y/o control", "ubicación de los puntos/zonas de medición y control", "ubicación de los puntos / zonas de medición y control"], 
    "parametros_a_medir": ["parámetros a medir", "parámetros que serán medidos", "parámetros a monitorear", "parámetros que serán utilizados para caracterizar el estado y evolución de las variables ambientales"],
    "limites_permitidos": ["límites permitidos/comprometidos", "límite permitido", "límites permitidos o comprometidos", "los límites considerados en la evaluación"],
    "duracion_monitoreo": ["duración del monitoreo"],
    "frecuencia_monitoreo": ["frecuencia del monitoreo"],
    "variable_ambiental": ["variable ambiental", "variables ambientales que serán objeto de muestreo, medición, análisis y/o control"],
    "duracion_frecuencia_medicion": ["duración y frecuencia de las actividades de muestreo, medición, análisis y/o control para cada parámetro", "duración y frecuencia de la medición"],
    "metodo_procedimiento_medicion": ["método o procedimiento de medición", "método o procedimiento de medición de cada parámetro", "método o procedimiento de muestreo, medición, análisis y/o control para cada parámetro"],
    "plazo_frecuencia_informe": ["plazo y frecuencia de entrega de informe", "plazo y frecuencia de entrega de informes", "período, frecuencia y plazo de entrega de los informes de seguimiento"],
    "destinatario": ["organismo (s) destinatario(s) de informes", "organismo destinatario de informes", "organismos destinatarios de informes"],
}


### Funciones para Tiempo

In [ ]:
def tic():
    global start_time
    start_time = time.time()

def toc():
    if 'start_time' in globals():
        elapsed_time = time.time() - start_time
        print(f"Elapsed time: {elapsed_time:.4f} seconds")
    else:
        print("Tic has not been called yet!")

# Funciones de Extracción de Tablas

### Funciones de extraer y limpiar

In [ ]:
def insertar_espacios(texto):
    texto = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', texto)
    texto = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', texto)
    texto = re.sub(r'(?<=\d)(?=[a-zA-Z])', ' ', texto)
    texto = re.sub(r',(?=\S)', ', ', texto)
    return texto

def limpiar_celda(celda):
    if isinstance(celda, str):
        celda = insertar_espacios(celda)
        return celda.replace('\r', ' ').strip().lower()
    return ""

def ajustar_titulos(tabla, numero_tabla):
    if all(col.startswith('Unnamed') for col in tabla.columns):
        if len(tabla) > 1:
            tabla.columns = tabla.iloc[0]
            tabla = tabla[1:]
    if 'Unnamed: 0' in tabla.columns:
        tabla.rename(columns={'Unnamed: 0': numero_tabla}, inplace=True)
    return tabla


def limpiar_tabla(tabla, numero_tabla):
    tabla = tabla.dropna(axis=1, how='all')
    tabla = tabla.dropna(how='all')
    for col in tabla.columns:
        tabla[col] = tabla[col].astype(object) 
        tabla.loc[:, col] = tabla[col].map(limpiar_celda).astype(object)
    tabla.columns = [limpiar_celda(col) for col in tabla.columns]
    tabla = ajustar_titulos(tabla, numero_tabla)
    return tabla

def extraer_y_limpiar_tablas(archivo_pdf):
    tables = tabula.io.read_pdf(archivo_pdf, pages='all', multiple_tables=True, lattice=True, encoding = 'latin-1', silent='True')
    all_tables = []
    for table in tables:
        if not table.empty and not (table.apply(lambda col: col.apply(lambda x: isinstance(x, str) and x.strip() == "")).all().all()):
            numero_tabla = detectar_numero_tabla(table)
            table = limpiar_tabla(table, numero_tabla)
            all_tables.append(table)
    return all_tables

def fusionar_tablas(all_tables):
    tablas_fusionadas = []
    i = 0

    while i < len(all_tables):
        tabla_actual = all_tables[i]
        numero_tabla_actual = detectar_numero_tabla(tabla_actual)
        tablas_a_fusionar = [tabla_actual]
        i += 1

        while i < len(all_tables):
            tabla_siguiente = all_tables[i]
            numero_tabla_siguiente = detectar_numero_tabla(tabla_siguiente)

            if numero_tabla_siguiente == numero_tabla_actual or (
                numero_tabla_siguiente == "no_identificado" and es_continuacion(tabla_actual, tabla_siguiente)
            ):
                tablas_a_fusionar.append(tabla_siguiente)
                i += 1
            else:
                break

        if len(tablas_a_fusionar) > 1:
            tabla_fusionada = pd.concat(tablas_a_fusionar, ignore_index=True)
            tablas_fusionadas.append(tabla_fusionada)
        else:
            tablas_fusionadas.append(tabla_actual)

    return tablas_fusionadas

def es_continuacion(tabla_anterior, tabla_actual):
    if len(tabla_actual.columns) <= len(tabla_anterior.columns):
        unnamed_columns = [col for col in tabla_actual.columns if 'unnamed' in str(col).lower()]
        if len(unnamed_columns) == len(tabla_actual.columns):
            return True
    return False

def renombrar_primera_columna(tablas_fusionadas):
    for idx, tabla in enumerate(tablas_fusionadas):
        if not tabla.empty and 'Unnamed: 0' in tabla.columns:
            tabla.rename(columns={'Unnamed: 0': 'campo'}, inplace=True)
            print(f"Primera columna de tabla {idx + 1} renombrada a 'campo'")
    return tablas_fusionadas

### Funciones detección de campos y números de tabla

In [ ]:
def detectar_numero_tabla(tabla):

    patrones = [
        r"(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])",
        r"(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])",
        r"(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.",
        r"(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.",
        r"n°(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])",
        r"n°(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])",
        r"tabla (?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])",
        r"tabla (?:[1-9]\d?|[1-9])"
    ]
    
    conceptos_dict = {
        "fase": ["fase del proyecto", "fase del proyecto (fase de aplicación de la medida)", "fases", "fase", "fases de ejecución", "fase de ejecución", "fase de cumplimiento", "fase del proyecto a la que aplica o en la que se dará cumplimiento", "fase del proyecto", "fase del proyecto a la que aplica", "fase en que se presenta"],
        "parte_asociada": ["parte, obra o acción a la que aplica", "parte, obra, acción, emisión, residuo o sustancias a la que aplica", "parte, obra, acción, emisión, residuo o sustancias a la que aplica", "relación con el proyecto", "parte, obra o acción que lo genera", "parte, obra o acción asociada", "parte, obra, acción, emisión, residuo o sustancias a las que aplica", "parte u obra a la que aplica", "emplazamiento, parte, obra o acción asociada"],
        "lugar_forma_oportunidad": ["lugar, forma y oportunidad de implementación", "lugar de verificación", "forma y oportunidad de implementación"],
        "lugar": ["lugar de implementación", "lugar de implementación de la medida"], 
        "forma_oportunidad": ["forma y oportunidad de implementación", "forma y oportunidad de implementación de la medida"],
        "metodo_implementacion": ["método de implementación de la medida"],    
        "forma_cumplimiento": ["forma de cumplimiento"],
        "tipo_medida": ["tipo de medida", "medida(s) asociada(s)", "medida asociada"], 
        "indicador_cumplimiento": ["indicador que acredita su cumplimiento", "indicador de cumplimiento de la medida", "indicador de cumplimiento", "indicador que acredite su cumplimiento", "método o procedimiento de medición de cada parámetro"],
        "control_seguimiento": ["forma de control y seguimiento", "seguimiento y control"],
        "ice": ["referencia al ice para mayor detalle", "referencia al ice para mayores detalles", "referencia al ice", "referencia al ice o documentos del expediente de evaluación que contenga la descripción detallada", "referencia a documentos del expediente de evaluación que contenga la descripción detallada", "referencia al ice para mayores detalles sobre este impacto específico"],
        "otros_legal": ["otros cuerpos legales", "otros cuerpos legales asociados"],
        "impacto": ["impacto asociado", "impacto ambiental", "impactos asociados", "impacto ambiental asociado"],
        "objetivo_descripcion_justificacion": ["objetivo, descripción y justificación", "objetivo del compromiso", "descripción del compromiso", "justificación del compromiso"],
        "objetivo": ["objetivo", "objetivo de la medida"], 
        "descripcion": ["descripción", "descripción de la medida"],
        "justificacion": ["justificación", "justificación de la medida"],
        "calificacion": ["calificación de la parte u obra"],
        "condiciones": ["condiciones o exigencias específicas del pronunciamiento", "condiciones o exigencias específicas para su otorgamiento"],
        "pronunciamiento": ["pronunciamiento del órgano competente"],
        "acciones": ["acciones o medidas a implementar para prevenir la contingencia", "acciones a implementar", "acciones o medida a implementar para controlar la emergencia"],
        "sma": ["oportunidad y vías de comunicación a la sma de la activación del plan de emergencia", "oportunidad y vías de comunicación a la sma de la activación del plan", "oportunidad y vías de comunicación a la sma", "activación del plan de emergencia"],
        "subcomponente": ["subcomponente", "subcomponente ambiental", "subcomponente ambiental objeto de seguimiento"],
        "medio_verificacion": ["medio de verificación"],
        "plazo_implementacion": ["plazo de implementación de la medida"],
        "indicador_exito": ["indicador de éxito", "indicador de efectividad de la medida"],
        "ubicacion": ["división político administrativa", "ubicación puntos de control", "ubicación de los puntos de control", "ubicación de los puntos o sitios de muestreo, medición, análisis y/o control", "ubicación de los puntos/zonas de medición y control", "ubicación de los puntos / zonas de medición y control"], 
        "parametros_a_medir": ["parámetros a medir", "parámetros que serán medidos", "parámetros a monitorear", "parámetros que serán utilizados para caracterizar el estado y evolución de las variables ambientales"],
        "limites_permitidos": ["límites permitidos/comprometidos", "límite permitido", "límites permitidos o comprometidos", "los límites considerados en la evaluación"],
        "duracion_monitoreo": ["duración del monitoreo"],
        "frecuencia_monitoreo": ["frecuencia del monitoreo"],
        "variable_ambiental": ["variable ambiental", "variables ambientales que serán objeto de muestreo, medición, análisis y/o control"],
        "duracion_frecuencia_medicion": ["duración y frecuencia de las actividades de muestreo, medición, análisis y/o control para cada parámetro", "duración y frecuencia de la medición"],
        "metodo_procedimiento_medicion": ["método o procedimiento de medición", "método o procedimiento de medición de cada parámetro", "método o procedimiento de muestreo, medición, análisis y/o control para cada parámetro"],
        "plazo_frecuencia_informe": ["plazo y frecuencia de entrega de informe", "plazo y frecuencia de entrega de informes", "período, frecuencia y plazo de entrega de los informes de seguimiento"],
        "destinatario": ["organismo (s) destinatario(s) de informes", "organismo destinatario de informes", "organismos destinatarios de informes"],
    }
    

    def validar_numero_tabla(numero_tabla):
        numero_tabla = re.sub(r"n°|tabla", "", numero_tabla, flags=re.IGNORECASE).strip()
        numero_tabla = numero_tabla.replace('.', '_')      
        match = re.search(r"(\d+)", numero_tabla)
        if match:
            numero = int(match.group(1))
            if numero > 12:
                return "no_identificado"
        return numero_tabla
        
    def contiene_excepciones(texto, patron,conceptos_dict):
        texto_anterior = re.search(rf"(\b\w+\b\s*){patron}", texto)
        texto_posterior = re.search(rf"{patron}(\s*\b\w+\b)", texto)
        if texto_anterior and re.search(r"\bnumeral\b", texto_anterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_anterior and re.search(r"\bpunto\b", texto_anterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_anterior and re.search(r"\bartículo\b", texto_anterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_anterior and re.search(r"\bley\b", texto_anterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_posterior and re.search(r"\bdel\s*ice\b", texto_posterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_posterior and re.search(r"[\\/]", texto_posterior.group(1), flags=re.IGNORECASE):
            return True
        if texto_posterior and re.search(r"\d{1,2}\.\d{1,2}\.\d{1,2}", texto_posterior.group(1), flags=re.IGNORECASE):
            return True
        for key, values in conceptos_dict.items():
            for value in values:
                if re.search(re.escape(value), texto, flags=re.IGNORECASE):
                    return True

        
        return False    

    numero_encontrado = "no_identificado"
    
    # Revisar los encabezados de las columnas
    for col in tabla.columns:        
        for patron in patrones:
            if contiene_excepciones(str(col).lower(), patron, conceptos_dict):
                return numero_encontrado
            match = re.findall(patron, str(col).lower())
            if match:
                if not contiene_excepciones(str(col).lower(), patron, conceptos_dict):
                    numero_encontrado = validar_numero_tabla(match[0])
                    if numero_encontrado != "no_identificado":
                        return numero_encontrado

    
    # Revisar las celdas de la primera fila (si existen)
    filas_a_revisar = min(len(tabla), 1)
    for i in range(filas_a_revisar):
        row = tabla.iloc[i]
        for cell in row:
            if cell:
                for patron in patrones:
                    match = re.findall(patron, str(cell).lower())
                    if match:
                        if not contiene_excepciones(str(cell).lower(), patron, conceptos_dict):
                            numero_encontrado = validar_numero_tabla(match[0])
                            if numero_encontrado != "no_identificado":
                                return numero_encontrado
        break
                            
    
    return numero_encontrado


def buscar_conceptos(tabla, conceptos):
    resultados = []
    
    # Crear un conjunto de conceptos sin espacios
    conceptos_sin_espacios = {concepto.replace(" ", "") for concepto in conceptos}
    
    # Buscar conceptos en los nombres de las columnas
    for j, column_name in enumerate(tabla.columns):
        column_name_str = str(column_name).replace(" ", "")
        
        if "norma" in conceptos_sin_espacios and column_name_str == "norma":
            if j + 1 < len(tabla.columns):
                contenido_derecha = tabla.columns[j + 1] if tabla.columns[j + 1] != "" else ""
            else:
                contenido_derecha = ""
            resultados.append((-1, j, contenido_derecha))
        
        elif any(concepto in column_name_str for concepto in conceptos_sin_espacios if concepto != "norma"):
            if j + 1 < len(tabla.columns):
                contenido_derecha = tabla.columns[j + 1] if tabla.columns[j + 1] != "" else ""
            else:
                contenido_derecha = ""
            resultados.append((-1, j, contenido_derecha))

    
    # Buscar conceptos en la primera columna de cada fila
    for i, row in tabla.iterrows():
        cell_str = str(row.iloc[0]).replace(" ", "")
        
        if any(concepto in cell_str for concepto in conceptos_sin_espacios):
            if len(row) > 1:
                contenido_derecha = row.iloc[1] if row.iloc[1] != "" else ""
            else:
                contenido_derecha = ""
            resultados.append((i, 0, contenido_derecha))
    
    return resultados

def concat_no_identificado(df, column_name='numero_tabla'):
    def is_concatenation_needed(prev_value, curr_value, next_value):
        if curr_value == "no_identificado":
            if (is_sequential(prev_value, next_value)):
                return True
        return False

    def is_sequential(prev_value, next_value):
        prev_parts = prev_value.split('_')
        next_parts = next_value.split('_')

        # Verifica que el último elemento de prev_parts y el primer elemento de next_parts sean secuenciales
        try:
            if len(prev_parts) == len(next_parts):
                return int(prev_parts[-1]) + 1 == int(next_parts[-1])
            elif len(prev_parts) + 1 == len(next_parts):
                return int(prev_parts[-1]) + 1 == int(next_parts[len(prev_parts) - 1])
            elif len(prev_parts) - 1 == len(next_parts):
                return int(prev_parts[len(next_parts) - 1]) + 1 == int(next_parts[-1])
        except ValueError:
            return False
        return False


    # Crear una lista para almacenar las filas actualizadas
    updated_rows = []

    i = 0
    while i < len(df):
        if i > 0 and i < len(df) - 1:
            prev_value = df.at[i - 1, column_name]
            curr_value = df.at[i, column_name]
            next_value = df.at[i + 1, column_name]

            if is_concatenation_needed(prev_value, curr_value, next_value):
                for col in df.columns:
                    if col != column_name: 
                        df.at[i - 1, col] = f"{df.at[i - 1, col]} {df.at[i, col]}".strip()

                i += 1
                continue
                
        
        updated_rows.append(df.iloc[i])
        i += 1

    # Crear un nuevo DataFrame con las filas actualizadas
    updated_df = pd.DataFrame(updated_rows, columns=df.columns)
    return updated_df

### Funciones aplicadas una vez creadas las tablas

In [ ]:
def concat_anterior(df):
    # Crear una lista para almacenar las filas a eliminar
    filas_a_eliminar = []

    # Recorrer el DataFrame por índice
    for i in range(1, len(df)):
        if df.at[i, 'numero_tabla'] == 'Concatenar a anterior':
            for col in df.columns:
                if col != 'numero_tabla':
                    df.at[i-1, col] = str(df.at[i-1, col]) + ' ' + str(df.at[i, col])
            filas_a_eliminar.append(i)
    df = df.drop(filas_a_eliminar)
    
    # Resetear el índice después de borrar filas
    df = df.reset_index(drop=True)
    columnas = ['numero_tabla', 'numero_tabla_original'] + [col for col in df.columns if col not in ['numero_tabla', 'numero_tabla_original']]
    df = df[columnas]
    
    return df


def in_between(df, column_name='numero_tabla'):
    df = df.reset_index(drop=True)
    
    def is_sequential(prev_value, next_value):
        prev_parts = prev_value.split('_')
        next_parts = next_value.split('_')

        try:
            # Convert last part of the split values to integers and compare
            if len(prev_parts) == len(next_parts):
                return int(next_parts[-1]) - int(prev_parts[-1]) == 2
            elif len(prev_parts) + 1 == len(next_parts):
                return int(next_parts[len(prev_parts) - 1]) - int(prev_parts[-1]) == 2
            elif len(prev_parts) - 1 == len(next_parts):
                return int(next_parts[-1]) - int(prev_parts[len(next_parts) - 1]) == 2
        except ValueError:
            return False
        return False

    def get_in_between_value(prev_value, next_value):
        prev_parts = prev_value.split('_')
        next_parts = next_value.split('_')

        if len(prev_parts) == len(next_parts):
            in_between_parts = prev_parts[:-1] + [str(int(prev_parts[-1]) + 1)]
        elif len(prev_parts) + 1 == len(next_parts):
            in_between_parts = prev_parts + [str(int(prev_parts[-1]) + 1)]
        elif len(prev_parts) - 1 == len(next_parts):
            in_between_parts = next_parts[:-1] + [str(int(next_parts[-1]) - 1)]
        
        return '_'.join(in_between_parts)

    for i in range(1, len(df) - 1):
        prev_value = df.at[i - 1, column_name]
        curr_value = df.at[i, column_name]
        next_value = df.at[i + 1, column_name]

        if curr_value == "no_identificado":
            if is_sequential(prev_value, next_value):
                print("in between")
                df.at[i, column_name] = get_in_between_value(prev_value, next_value)

    return df

# Eliminar todos los archivos en las carpetas de origen y destino
def limpiar_carpetas(carpeta):
    for filename in os.listdir(carpeta):
        file_path = os.path.join(carpeta, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error al eliminar el archivo {file_path}: {e}")

### Función de Generación de Tablas

In [ ]:
def last_check_merge(df):
    # Inicializamos una columna con True para las filas que no tienen siguiente fila
    df['valid'] = True
    df['non_empty_cells'] = df.apply(lambda row: row.apply(lambda x: pd.notna(x) and str(x).strip() != "").sum(), axis=1)
    for i in range(len(df) - 1):
        current_char = df.iloc[i]['numero_tabla'][0]
        next_char = df.iloc[i + 1]['numero_tabla'][0]
        if current_char.isdigit() and next_char.isdigit():
            if next_char < current_char and not (current_char == '9' and next_char == '1'):
                df.iloc[i + 1, df.columns.get_loc('valid')] = False
            else:
                df.iloc[i + 1, df.columns.get_loc('valid')] = True

    # Fusionar filas no válidas con la fila anterior o siguiente según el valor de non_empty_cells
    rows_to_drop = []
    for i in range(1, len(df)):
        if df.iloc[i]['valid'] == False:
            if i < len(df) - 1 and df.iloc[i - 1]['non_empty_cells'] > df.iloc[i + 1]['non_empty_cells'] and df.iloc[i + 1]['numero_tabla'] != 'no_identificado':
                # Fusionar con la fila siguiente
                for col in df.columns:
                    if col != 'numero_tabla' and col != 'valid' and col != 'non_empty_cells':
                        df.iloc[i + 1, df.columns.get_loc(col)] += " " + df.iloc[i][col]
                print(f"last check merge {df.iloc[i]['numero_tabla_original']}")
                rows_to_drop.append(i)
            else:
                # Fusionar con la fila anterior
                for col in df.columns:
                    if col != 'numero_tabla' and col != 'valid' and col != 'non_empty_cells':
                        df.iloc[i - 1, df.columns.get_loc(col)] += " " + df.iloc[i][col]
                print(f"last check merge {df.iloc[i]['numero_tabla_original']}")
                rows_to_drop.append(i)
                
    
    # Eliminar filas no válidas
    df.drop(rows_to_drop, inplace=True)
    df.drop(columns=['valid', 'non_empty_cells'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
def extraer_tablas(archivo_destino, conceptos_dict, cell):

    # --------------------------------------
    # EXTRAER TABLAS DE INTERES
    # --------------------------------------
        
    # Extraer y limpiar tablas
    all_tables = extraer_y_limpiar_tablas(archivo_destino)
    print("-tablas extraidas-")
    # Fusionar tablas si es necesario
    tablas_fusionadas = fusionar_tablas(all_tables)

    # Renombrar la primera columna si es necesario
    tablas_finales = renombrar_primera_columna(tablas_fusionadas)

    # Revisar títulos y contenido de las tablas
    tables_over_6 = []
    encontrada_tabla_6 = False
    o_count = 0
    x_count = 0

    for i, table in enumerate(tablas_finales, start=1):
        numero_tabla = detectar_numero_tabla(table)
        if re.match(r'^[6-9][0-9]?_', numero_tabla) and not encontrada_tabla_6:
            encontrada_tabla_6 = True
            print("encontrada")
        if encontrada_tabla_6:
            tables_over_6.append(table)

    # --------------------------------------
    # ANÁLISIS TABLA POR TABLA
    # --------------------------------------

    # Crear un DataFrame vacío con las columnas para cada grupo de conceptos
    columnas_dict = list(conceptos_dict.keys())
    columnas = list(set(columnas_dict))
    data = {columna: [] for columna in columnas}
    data["numero_tabla"] = []
    data["numero_tabla_original"] = []
    ultima_fila_primera_columna = None
    numero_tabla_list = []
    
    print(f"-comienza analisis {cell}-, lenght: {len(tables_over_6)}")
    
    for i, table in enumerate(tables_over_6, start=1):
        try: 
            numero_tabla = detectar_numero_tabla(table).replace('.', '_')
            filas = len(table)
            num_columnas = len(table.columns) if filas > 0 else 0
            
            fila_data = {columna: "" for columna in columnas}
            fila_data["numero_tabla"] = numero_tabla
            fila_data["numero_tabla_original"] = str(i)
    
            # Para Unir Tablas Comprobar las condiciones al inicio de la iteración
            if (
                ultima_fila_primera_columna is not None and
                filas > 0 and
                ('unnamed' in table.columns[0]) and
                table.columns[1] != ""
            ):
                table.columns = [ultima_fila_primera_columna if col.startswith('unnamed') else col for col in table.columns]
        
            conceptos_actuales = conceptos_dict
            for concepto_nombre, conceptos in conceptos_actuales.items():
                resultados = buscar_conceptos(table, conceptos)
                if resultados:            
                    for (fila, columna, contenido) in resultados:
                        o_count += 1
                        fila_data[concepto_nombre] = contenido
                        if (fila == -1) and (concepto_nombre != "componente_materia") and (concepto_nombre != "nombre_compromiso") and (concepto_nombre != "riesgo_contingencia"):
                            fila_data["numero_tabla"] = "Concatenar a anterior"
                            numero_tabla = "Concatenar a anterior"
                else:
                    x_count += 1
    
            for columna in columnas:
                data[columna].append(fila_data[columna])
            data["numero_tabla"].append(fila_data["numero_tabla"])
            data["numero_tabla_original"].append(fila_data["numero_tabla_original"])
            numero_tabla_list.append(numero_tabla)
            if filas > 0 and num_columnas > 0:
                ultima_fila_primera_columna = table.iloc[-1, 0]
        except: 
            print(f"error en análisis tabla por tabla: {numero_tabla}, {i}/{len(tables_over_6)}") 

    # --------------------------------------
    # CREAR DF, CONCATENAR Y RENOMBRAR
    # --------------------------------------
    try: 
        print(f"-continua analisis {cell}-")
        # Crear el DataFrame
        df = pd.DataFrame(data)
        o_percent = (o_count / (o_count + x_count)) * 100 if o_count > 0 and x_count > 0 else 0
        print(o_percent)
    
        df = concat_anterior(df)
        df = concat_no_identificado(df)
        df = in_between(df)
        df = last_check_merge(df)
    
        # Eliminar lo innecesario
        df.replace({"": None, " ": None}, inplace=True)
        cols_to_check = df.columns.difference(['numero_tabla', 'numero_tabla_original'])
        rows_to_remove = df[cols_to_check].isnull().all(axis=1)
        df = df.drop(df[rows_to_remove].index)
        df.replace({None: ""}, inplace=True)
    except: 
        print(f"error en crear, concatenar y renombrar df") 
    # --------------------------------------
    # ANÁLISIS POR FILA DEL DF
    # --------------------------------------
    try: 
        for index, row in df.iterrows():
            non_empty_cells = row.apply(lambda x: pd.notna(x) and str(x).strip() != "").sum()
            percentage_non_empty = (non_empty_cells / len(row)) * 100 if len(row) != 0  else 0 
            print(f"Fila {index}: {percentage_non_empty:.2f}% celdas,  {non_empty_cells} campos identificados,  {row['numero_tabla']}, {row['numero_tabla_original']}")

        # Calcular porcentaje de filas cuyo valor en numero_tabla sea "No Identificado"
        total_rows = len(df)
        unidentified_rows = df['numero_tabla'].apply(lambda x: x.strip().lower() == "no_identificado").sum()
        percentage_unidentified = (unidentified_rows / total_rows) * 100 if total_rows != 0 else 0
        print(f"Porcentaje de filas con 'no_identificado' en numero_tabla: {percentage_unidentified:.2f}%")


    except: 
        print(f"error en revision de filas")
    return df, tables_over_6, percentage_unidentified

# Definir Funciones de Separación de Secciones

### Definir funciones de Búsqueda de Conceptos

In [ ]:
def fix_encoding(text):
    # Intenta diferentes correcciones de codificación
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        try:
            return text.encode('windows-1252').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            return text  # Si no hay una solución, devuelve el texto original

def extract_pdf_text(file_name):
    with pdfplumber.open(file_name) as pdf:
        text = ""
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                page_text = fix_encoding(page_text)
                text += page_text + "\n"

    return text

    
def find_fuzzy_text(target_text, document_text, window_size=100, threshold=85):
    matches = []
    doc_length = len(document_text)
    window_size = len(target_text) + 50
    
    for i in range(0, doc_length - window_size + 1, window_size // 2):
        segment = document_text[i:i + window_size]
        similarity = fuzz.partial_ratio(target_text, segment)
        if similarity >= threshold:
            exact_start = find_exact_position(target_text, segment)
            matches.append((i + exact_start, similarity, segment[exact_start:exact_start + len(target_text)]))

    for match in matches:
        if target_text != " ":
            pass

    if not matches:
        matches.append((-1, 0, "No encontrado"))

    return matches

def find_exact_position(target_text, segment):
    best_match_start = 0
    best_match_score = 0

    for start in range(len(segment) - len(target_text) + 1):
        match_score = fuzz.ratio(target_text, segment[start:start + len(target_text)])
        if match_score > best_match_score:
            best_match_score = match_score
            best_match_start = start
    
    return best_match_start

### Definir funciones de separación de secciones

In [ ]:
def extract_sections(document_text, keywords):
    sections = {}
    matches = []
    
    for key, texts in keywords.items():
        best_match = (-1, 0, "")
        
        if isinstance(texts, list):
            for text in texts:
                result = find_fuzzy_text(text, document_text, window_size=200)
                for match in result:
                    if match[1] > best_match[1] or (match[1] == best_match[1] and match[0] < best_match[0]):
                        best_match = match
                if best_match[0] != -1:
                    break
        else:
            result = find_fuzzy_text(texts, document_text, window_size=200)
            for match in result:
                if match[1] > best_match[1] or (match[1] == best_match[1] and match[0] < best_match[0]):
                    best_match = match

        if best_match[0] != -1:
            matches.append((best_match[0], key, best_match[1]))

    matches.sort()
    
    last_index = 0
    for i, (index, key, score) in enumerate(matches):
        if i > 0:
            prev_key = matches[i-1][1]
            sections[prev_key] = document_text[last_index:index].strip()
        last_index = index

    if matches:
        last_key = matches[-1][1]
        sections[last_key] = document_text[last_index:].strip()
    else:
        sections["final"] = document_text.strip()

    return sections

def separador_secciones(document_text):   
    # Define las palabras clave y los textos de búsqueda
    keywords = {
        "inicial": " ",
        "vistos": "VISTOS",
        "considerando": ["CONSIDERANDO","CONSIDERANDO:", 
            "Que, en lo relativo a los efectos, características y circunstancias señalados en el artículo 11 de la Ley Nº 19.300",
            "Que, en lo relativo a los efectos, características y circunstancias señalados en el artículo 11 de la Ley Nº19.300",
            'Que en lo relativo a los efectos características y circunstancias señalados en los literales "a","b", "c", "d", "e" y "f" del artículo 11 de la Ley 19.300',
            "Que, en lo relativo a los efectos, características y circunstancias señalados ene l artículo 11 de la Ley 19.300",
        ],
        "mitigacion": [
            "Que, del proceso de evaluación de impacto ambiental del Proyecto puede concluirse que las siguientes medidas de mitigación", 
            "Que, del proceso de evaluación de impacto ambiental del Proyecto puede concluirse que la siguiente medida de mitigación",
            "Que, las medidas de mitigación, reparación y/o compensación asociadas a los efectos, características y circunstancias del artículo 11 de la Ley N° 19.300",
            "Plan de medidas asociado al Medio Humano",
            "Plan de Medidas de Mitigación, Compensación y Reparación",
            "Medidas de mitigación, reparación y/o compensación",
        ],
        "pas": [
            "Que resultan aplicables al Proyecto los siguientes permisos ambientales sectoriales, asociados a las correspondientes partes, obras o acciones que se señalan a continuación",
            "Que resultan aplicables al Proyecto el siguiente permiso ambiental sectorial",  
            "Que resultan aplicables al Proyecto los siguientes permisos ambientales sectoriales",
            "el proyecto ha obtenido los permisos de carácter ambiental de los siguientes artículos"
            "Que al proyecto le resultan aplicables los permisos ambientales sectoriales de los artículos",
            "Que, al proyecto le es aplicable los siguientes permisos ambientales sectoriales señalados en el Título VII del Reglamento del SEIA",
            "Que al proyecto no le resultan aplicables permisos ambientales sectoriales, asociados a las correspondientes partes, obras o acciones.",
            "Permisos Ambientales Sectoriales de Contenido Únicamente Ambiental",
            "Permisos ambientales sectoriales mixtos", 
            "que resultan aplicables al Proyecto, asociados a las correspondientes partes, obras o acciones que se señalan a continuación",
            "requiere de los permisos ambientales sectoriales contemplados en los artículos",
            "PERMISOS Y PRONUNCIAMIENTO AMBIENTALES SECTORIALES",
            "PERMISOS AMBIENTALES SECTORIALES APLICABLES", 
            "PERMISOS AMBIENTALES SECTORIALES",
            "Permiso Ambientales Sectoriales (PAS)",
            "Que tenidos a la vista los antecedentes del proyecto y sus características, se concluye que el proyecto requiere de los permisos ambientales sectoriales que se enlistan en los artículos",
            "Que dada las características del proyecto no le son aplicables permisos ambientales sectoriales",
            "éste requiere para su ejecución los permisos de caracter ambiental se los siguientes artículos",
            "El Proyecto no requiere de ningún permiso, ni pronunciamiento ambiental sectorial de acuerdo al Título VII del Párrafo",
        ],
        "forma_cumplimiento": [ 
            "Que, de acuerdo a los antecedentes que constan en el expediente de evaluación, la forma de cumplimiento de la normativa de carácter ambiental",
            "Que, de acuerdo con los antecedentes que constan en el expediente de evaluación, la forma de cumplimiento de la normativa de carácter ambiental",
            "Que, de acuerdo con antecedentes que constan en el expediente de evaluación, la forma de cumplimiento de la normativa de carácter ambiental",
            "Que, de acuerdo a los antecedentes que constan en el expediente de evaluacion, la forma de cumplimiento de lai normativa de carâcter ambiental",
            "Que, de acuerdo a los antecedentes que constan en el expediente de evaluación, la forma de cumplimiento de la normava de carácter ambiental",           
            "la forma de cumplimiento de la normativa de carácter ambiental aplicable al Proyecto es la siguiente",
            "la forma de cumplimiento de la normativa de carácter ambiental aplicable al Proyecto",
            "Que con relación al cumplimiento de la legislación aplicable, el titular acredita el cumplimiento a dichas normas",
            "Normas relacionadas con las partes, obras, actividades o acciones, emisiones, residuos y sustancias peligrosas del proyecto",
            "Normas relacionadas al emplazamiento del proyecto",
            "Componente/materia:",
            "NORMATIVA DE CARÁCTER AMBIENTAL APLICABLE",
            "Que, en relación con el cumplimiento de la normativa ambiental aplicable al proyecto",
            "Normativa Ambiental Sectorial",
        ],
        "condiciones_exigencias": [
            'Que, La Comisión de Evaluación estableción las siguientes condiciones o exigencias para la aprobación del Proyecto', 
            'Que, para ejecutar el Proyecto deben cumplirse las siguientes condiciones o exigencias, en concordancia con el artículo 25 de la Ley N° 19.300',
            'Que, durante el procedimiento de evaluación de la DIA se estableció que el Titular del proyecto deberá cumplir con lo siguiente',
            'Que, durante el procedimiento de evaluación de la DIA se establecieron las siguientes condiciones al Proyecto:',
            'Que, en uso de sus facultades, la Comisión de Evaluación, ha resuelto imponer al titular, como condiciones o exigencias',
            "Que, para ejecutar el Proyecto se requieren condiciones o exigencias especiales, en concordancia con el artículo 25 de la Ley N° 19.300", 
            "Las condiciones o exigencias para ejecutar el proyecto son las siguientes", 
            "El Proyecto contará con las siguientes condiciones y/o exigencias",
            "Que, durante el procedimiento de evaluación de la DIA se establecieron condiciones o exigencias adicionales al Proyecto",
            'Durante el proceso de evaluación fueron solicitadas las siguientes exigencias para el desarrollo del Proyecto',
            'Durante el proceso de evaluación fueron solicitadas las siguientes condiciones para el desarrollo del Proyecto',
            "Que, las condiciones o exigencias, en concordancia con el artículo 25 de la Ley N° 19.300, corresponden a",
            'para ejecutar el Proyecto deben cumplirse las siguientes condiciones o exigencias',
            'estableció las siguientes condiciones o exigencias para la aprobación del Proyecto',
            'se establecieron las siguientes condiciones o exigencias al Proyecto',
            ' CONDICIONES Y EXIGENCIAS',
            "pueda ejecutarse, necesariamente deberá cumplir con todas las normas vigentes que le sean aplicables",
            "Propoisición de condiciones o exigencias específicas que el titular debe cumplir para ejecutar el Proyecto",
        ],
        "compromisos_voluntarios": [
            "Que, durante el procedimiento de evaluación de la DIA el Titular del Proyecto propuso los siguientes compromisos ambientales voluntarios",
            "Que, durante el procedimiento de evaluación de la DIA el Titular se ha obligado voluntariamente a los siguientes compromisos ambientales",
            "Que, durante el procedimiento de evaluación del EIA el Titular se ha obligado voluntariamente a los siguientes compromisos ambientales",
            "Que, durante el procedimiento de evaluación del EIA el Titular del Proyecto propuso los siguientes compromisos ambientales voluntarios",
            'Que, durante el procedimiento de evaluación de la DIA el Titular del Proyecto propuso el siguiente compromiso ambiental voluntario:',
            "Que, durante el procedimiento de evaluación de la DIA se establecieron los siguientes compromisos ambientales voluntarios", 
            "Que, durante el procedimiento de evaluación del EIA se establecieron los siguientes compromisos ambientales voluntarios", 
            "Que, durante el procedimiento de evaluación el Titular del Proyecto, en cuanto al establecimiento de compromisos ambientales voluntarios, indicó lo siguiente:",
            "el titular se ha comprometido voluntariamente a lo siguiente",
            "Que, el proyecto considera los siguientes compromisos ambientales voluntarios:", 
            "Que, durante el procedimiento de evaluación de la DIA el Titular del Proyecto propuso los siguientes compromisos ambientales voluntarios",
            "Que, para ejecutar el Proyecto deben cumplirse los siguientes compromisos voluntarios, condiciones o exigencias:", 
            "Que, el proyecto contará con los siguientes compromisos voluntarios",
            "El Proyecto contará con el siguiente compromiso voluntario", 
            "9. Compromisos Ambientales Voluntarios",
            "el titular adquirió el siguiente compromiso ambiental voluntario:",
            "COMPROMISOS AMBIENTALES VOLUNTARIOS", "COMPROMISOS VOLUNTARIOS", "Compromiso ambiental voluntario"
        ],  
        "contingencias_emergencias": [ 
            "Que, las medidas relevantes del plan de contingencia se detallan acciones a seguir",
            "Que, las medidas relevantes del Plan de Prevención de Contingencias",
            "Que, las medidas o acciones relevantes del plan de prevención de contingencias",
            "Que, las medidas relevantes del Plan de Prevenciôn de Contingencias y del Plan de Emergencias",
            'Que, el proyecto contará con un "Plan de Prevencion de Contingencias"', 
            "Que, las medidas relevantes del Plan de Prevención de Emergencias",
            ". PLAN DE PREVENCIÓN DE CONTINGENCIAS",
            'Que, respecto del Plan de Prevención de Contingencias',
            "Planes Consolidados de Contingencia y Emergencia", 
            ". PLAN DE EMERGENCIAS",
            "Riesgo o contingencia", 
            "Que, toda vez que ocurra una contingencia, en alguna de las estructuras, partes o sistemas",
            "Medidas de prevención de riesgos y control de accidentes",
            "Que, en caso de emergencias producidas por materiales o sustancia peligrosas, o que puedan afectar, pudiendo ser o no alguna de las establecidas en los planes de contingencia",
        ],
        "plan_seguimiento":[
            "Que, el plan de seguimiento de las variables ambientales relevantes que dieron origen al EIA es el siguiente:",
            "Que, durante la evaluación ambiental del EIA y sus Adendas se estableció el siguiente Plan de Seguimiento Ambiental",
            "Que, el plan de seguimiento de las variables ambientales relevantes que fueron objeto de evaluación ambiental es el siguiente:",
            "8. Plan de seguimiento ambiental",
        ],
        "participacion_ciudadana": [
            "Evaluación técnica de las observaciones ciudadanas:"
            "Que, no se solicitó la apertura de proceso de participación ciudadana, conforme a lo dispuesto en el artículo",
            "Que, durante el proceso de participación ciudadana, desarrollado conforme a lo dispuesto en el artículo",
            "Que, durante el proceso de evaluación no hubo proceso de participación ciudadana (PCA), conforme a lo dispuesto en el artículo 30 bis de la Ley N° 19.300.",
            "Que, respecto a la apertura de proceso de participación ciudadana, conforme a lo dispuesto en el artículo",
            "durante el proceso de participación ciudadana, desarrollado conforme a lo dispuesto en el artículo",
            "Síntesis del proceso de participación",
            "Las observaciones que no cumplen con los requisitos establecidos en el artículo 29 de la Ley N° 19.300 y en el artículo 90 del Reglamento del SEIA son las siguientes",
            "14.1 Admisibilidad de las observaciones ciudadanas",
        ],
        "pronunciamiento": [
            "de conformidad con lo dispuesto en el artículo 161 del Reglamento del SEIA",
            "emitió el pronunciamiento a que se refiere el artículo 4.14.2 de la Ordenanza General de Urbanismo y Construcciones"
        ],
        "pueblos_indigenas": [
            "en la presente evaluación no se realizaron reuniones con grupos humanos pertenecientes a pueblos indígenas del artículo 86 del Reglamento del SEIA",
            "acta de la reunión realizada con grupos humanos pertenecientes a pueblos indígenas localizados en el área en que se desarrollará el Proyecto.",
        ],
        "remitir": [
            "el Titular deberá remitir a la Superintendencia del Medio Ambiente la información respecto de las condiciones, compromisos o medidas", 
            "Que, el Titular deberá remitir a la Superintendencia del Medio Ambiente la información respecto de las condiciones, compromisos o medidas, ya sea por medio de monitoreos, mediciones, reportes",
        ],
        "informar_ejecucion": [
            "el Titular deberá informar a la Superintendencia del Medio Ambiente la realización de la gestión, acto o faena mínima que da cuenta del inicio de la ejecución de obras", 
            "Que, el Titular deberá informar a la Superintendencia del Medio Ambiente la realización de la gestión, acto o faena mínima que da cuenta del inicio de la ejecución de obras",
            "con el objeto de dar adecuado seguimiento a la ejecución del Proyecto, el Titular deberá informar",
        ],
        "considerando_otros": [
            "Que, la Superintendencia del Medio Ambiente, de oficio o a petición de parte o de algún organismo sectorial, podrá aprobar, modificar o complementar el contenido del plan de seguimiento de las variables ambientales",
            "fue publicada en el Diario Oficial de la República de Chile con fecha",
            "que cuentan con resolución de calificación ambiental favorable y permiten explotar y operar las distintas instalaciones que componen la misma",
        ],
        "resuelvo": ["RESUELVO","RESUELVO:", "RESUELVE:"]
    }

    # Eliminación Manual de un caso específico
    if "Ampliación Centro Logístico Temuco" in document_text:
        keywords.pop("compromisos_voluntarios", None) 
        print("Keywords Ampliación Centro Logístico Temuco Removidos")

    # Define las palabras clave y los textos de búsqueda
    sections = extract_sections(document_text, keywords)
    
    # Asegurar que existan 9 secciones
    if len(sections) == 10:
        print("TODO OK")
    else: 
        print(f"NO HAY 10 SECCIONES, hay {len(sections)}")
        
        # Lista de claves requeridas
        required_keys = ['mitigacion', 'plan_seguimiento', 'pas', 'forma_cumplimiento', 'compromisos_voluntarios', 'condiciones_exigencias', 'contingencias_emergencias']
    
        # Verificar si las claves requeridas están presentes en el diccionario
        missing_keys = [key for key in required_keys if key not in sections]
        
        # Imprimir las claves faltantes
        if missing_keys:
            print("Las siguientes claves faltan en el diccionario:", missing_keys)

    return sections

### Función Separación de las Tablas Identificadas según su sección

In [ ]:
def separacion_tablas_secciones(df, sections_text_keys):

    df_pas = pd.DataFrame(columns=df.columns)
    df_mitigacion = pd.DataFrame(columns=df.columns)
    df_pas = pd.DataFrame(columns=df.columns)
    df_mitigacion = pd.DataFrame(columns=df.columns)
    df_plan_seguimiento = pd.DataFrame(columns=df.columns)
    df_forma_cumplimiento = pd.DataFrame(columns=df.columns)
    df_compromiso_voluntario = pd.DataFrame(columns=df.columns)
    df_condiciones_exigencias = pd.DataFrame(columns=df.columns)
    df_contingencias_emergencias = pd.DataFrame(columns=df.columns)  
    df_no_identificados = pd.DataFrame(columns=df.columns)  
    
    for column in list(conceptos_dict.keys()):
        if column not in df.columns:
            df[column] = ""

    # Asegurar presencia
    if "forma_cumplimiento" not in sections_text_keys:
        print("Advertencia: La sección principal 'forma_cumplimiento' no está presente en sections_text_keys.")
    # Preprocesamiento
    df['numero_tabla'] = df['numero_tabla'].str.replace('_', '.')
    df= df.drop(columns=["numero_tabla_original"])

    # Crear DataFrame PAS
    if "pas" in sections_text_keys:
        df_pas = df.loc[(df['pronunciamiento'] != '')]
        df = df.loc[(df['pronunciamiento'] == '')]
    else:
        df_pas = pd.DataFrame(columns=df.columns)

    # Crear DataFrame de forma_cumplimiento
    if "forma_cumplimiento" in sections_text_keys:
        df_forma_cumplimiento = df.loc[(df['forma_cumplimiento'] != '')]
        df = df.loc[(df['forma_cumplimiento'] == '')]
    else:
        df_forma_cumplimiento = pd.DataFrame(columns=df.columns)

    # Crear DataFrame de mitigacion
    if "mitigacion" in sections_text_keys:
        df_mitigacion = df.loc[(df['medio_verificacion'] != '') | (df['tipo_medida'] != '')]
        df = df.loc[(df['medio_verificacion'] == '') & (df['tipo_medida'] == '')]
    else:
        df_mitigacion = pd.DataFrame(columns=df.columns)

    # Crear DataFrame de plan_seguimiento
    if "plan_seguimiento" in sections_text_keys:
        df_plan_seguimiento = df.loc[
            (df['plazo_implementacion'] != '') | 
            (df['indicador_exito'] != '') | 
            (df['ubicacion'] != '') | 
            (df['parametros_a_medir'] != '') | 
            (df['limites_permitidos'] != '') | 
            (df['duracion_monitoreo'] != '') | 
            (df['frecuencia_monitoreo'] != '') | 
            (df['variable_ambiental'] != '') | 
            (df['duracion_frecuencia_medicion'] != '') | 
            (df['metodo_procedimiento_medicion'] != '') | 
            (df['plazo_frecuencia_informe'] != '') | 
            (df['destinatario'] != '')
        ]
        df = df.loc[
            (df['plazo_implementacion'] == '') & 
            (df['indicador_exito'] == '') & 
            (df['ubicacion'] == '') & 
            (df['parametros_a_medir'] == '') & 
            (df['limites_permitidos'] == '') & 
            (df['duracion_monitoreo'] == '') & 
            (df['frecuencia_monitoreo'] == '') & 
            (df['variable_ambiental'] == '') & 
            (df['duracion_frecuencia_medicion'] == '') & 
            (df['metodo_procedimiento_medicion'] == '') & 
            (df['plazo_frecuencia_informe'] == '') & 
            (df['destinatario'] == '')
        ]
    else:
        df_plan_seguimiento = pd.DataFrame(columns=df.columns)

    # Crear DataFrame de contingencias_emergencias
    if "contingencias_emergencias" in sections_text_keys:
        df_contingencias_emergencias = df.loc[(df['acciones'] != '') | (df['sma'] != '') | (df['riesgo_contingencia'] != '')]
        df = df.loc[(df['acciones'] == '') & (df['sma'] == '') & (df['riesgo_contingencia'] == '')]
    else:
        df_contingencias_emergencias = pd.DataFrame(columns=df.columns)
    
    
    if "compromisos_voluntarios" in sections_text_keys and "sin_compromisos_voluntarios" not in sections_text_keys:
        df_compromiso_voluntario = df.loc[(df['nombre_compromiso'] != '')]
        df = df.loc[(df['nombre_compromiso'] == '')]
    
    # Voluntario
    if "compromisos_voluntarios" in sections_text_keys and "sin_compromisos_voluntarios" not in sections_text_keys:
        voluntario_keywords = ['voluntario', 'voluntaria', 'voluntarios', 'CV', 'CAV', 'CV-COM']

        def contains_keyword_fuzzy(row, keywords, threshold=80):
            for cell in row:
                for keyword in keywords:
                    if fuzz.partial_ratio(str(cell).lower(), keyword.lower()) >= threshold:
                        return True
            return False

        voluntario_rows = [row for _, row in df.iterrows() if contains_keyword_fuzzy(row, voluntario_keywords)]
        df_compromiso_voluntario = pd.DataFrame(voluntario_rows, columns=df.columns)
        df = df.drop(df_compromiso_voluntario.index, errors='ignore')

    if "condiciones_exigencias" in sections_text_keys and "sin_condiciones_exigencias" not in sections_text_keys:
        condiciones_keywords = ['condiciones', 'exigencias', 'exigencia']

        def contains_keyword_fuzzy(row, keywords, threshold=80):
            for cell in row:
                for keyword in keywords:
                    if fuzz.partial_ratio(str(cell).lower(), keyword.lower()) >= threshold:
                        return True
            return False

        condiciones_rows = [row for _, row in df.iterrows() if contains_keyword_fuzzy(row, condiciones_keywords)]
        df_condiciones_exigencias = pd.DataFrame(condiciones_rows, columns=df.columns)
        df = df.drop(df_condiciones_exigencias.index, errors='ignore')
    

    # Procesar no identificados En base a presencia en el textos 
    def classify_tables(df, sections_text_keys, df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados):
        """
        Clasifica las tablas priorizando la presencia de "sin_" términos, luego evalúa otros casos.
        Si ambos "sin_" están presentes, las filas se descartan.
        """
        if df.empty:
            return df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados
        
        # Variables para facilitar la lógica
        tiene_condiciones = "condiciones_exigencias" in sections_text_keys
        tiene_compromisos = "compromisos_voluntarios" in sections_text_keys
        sin_condiciones = "sin_condiciones_exigencias" in sections_text_keys
        sin_compromisos = "sin_compromisos_voluntarios" in sections_text_keys
        
        # PRIMER NIVEL: Evaluación de "sin_" términos
        
        # Caso especial: Si tiene ambos "sin_", descartar filas
        if sin_condiciones and sin_compromisos:
            print(f"Se descartan {df.shape[0]} filas por tener ambos sin_")
            return df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados
        
        # Caso 1: Si tiene "sin_condiciones", debe ir a compromisos
        if sin_condiciones and df_compromiso_voluntario.empty:
            print(f"No Identificados asignados a Compromisos Voluntarios por sin_condiciones: {df.shape[0]} filas")
            df_compromiso_voluntario = pd.concat([df_compromiso_voluntario, df], ignore_index=True)
            return df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados
        
        # Caso 2: Si tiene "sin_compromisos", debe ir a condiciones
        if sin_compromisos and df_condiciones_exigencias.empty:
            print(f"No Identificados asignados a Condiciones y Exigencias por sin_compromisos: {df.shape[0]} filas")
            df_condiciones_exigencias = pd.concat([df_condiciones_exigencias, df], ignore_index=True)
            return df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados
        
        # SEGUNDO NIVEL: Evaluación de términos principales
        
        # Caso 3: Ambos presentes sin "sin_" - No asignar
        if tiene_condiciones and tiene_compromisos:
            print("Tablas no identificadas no asignadas porque cumplen ambas condiciones simultáneamente.")
            df_no_identificados = pd.concat([df_no_identificados, df], ignore_index=True)
        
        # Caso 4: Solo condiciones presente
        elif tiene_condiciones and df_condiciones_exigencias.empty:
            print(f"No Identificados asignados a Condiciones y Exigencias: {df.shape[0]} filas")
            df_condiciones_exigencias = pd.concat([df_condiciones_exigencias, df], ignore_index=True)
        
        # Caso 5: Solo compromisos presente
        elif tiene_compromisos and df_compromiso_voluntario.empty:
            print(f"No Identificados asignados a Compromisos Voluntarios: {df.shape[0]} filas")
            df_compromiso_voluntario = pd.concat([df_compromiso_voluntario, df], ignore_index=True)
        
        # Caso 6: Ninguno presente
        else:
            print("Tablas no identificadas no asignadas porque ninguna se identificó en el texto.")
            df_no_identificados = pd.concat([df_no_identificados, df], ignore_index=True)
        
        return df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados

    df_condiciones_exigencias, df_compromiso_voluntario, df_no_identificados = classify_tables(
        df=df,
        sections_text_keys=sections_text_keys,
        df_condiciones_exigencias=df_condiciones_exigencias,
        df_compromiso_voluntario=df_compromiso_voluntario,
        df_no_identificados=df_no_identificados
    )
    
    # Procesar no identificados En base a largo de 
    if df_no_identificados.shape[0] != 0: 
        print("Alternativa 2: Asignando no identificadas según número de identificadas")
        df = df[(df['numero_tabla'] != 'no.identificado') & (~df['numero_tabla'].str[0].isin(['0', '1', '2', '3', '4', '5']))]
        # Caso 7: No hay tablas de compromisos asignadas
        if (df_compromiso_voluntario.shape[0] == 0) & (df_condiciones_exigencias.shape[0] > 0) & ("compromisos_voluntarios" in sections_text_keys): 
            df_compromiso_voluntario = pd.concat([df_compromiso_voluntario, df], ignore_index=True)
            print(f"No Identificados {df.shape[0]} asignados a Compromisos Voluntarios: {df_compromiso_voluntario.shape[0]}, {df.numero_tabla}")

        # Caso 8: No hay tablas de condiciones asignadas
        elif (df_condiciones_exigencias.shape[0] == 0) & (df_compromiso_voluntario.shape[0] > 0) & ("condiciones_exigencias" in sections_text_keys):
            df_condiciones_exigencias = pd.concat([df_condiciones_exigencias, df], ignore_index=True)
            print(f"No Identificados {df.shape[0]} asignados a Condiciones y Exigencias: {df_condiciones_exigencias.shape[0]}, {df.numero_tabla} ")

        # Caso 9: Hay tablas de ambos asignadas y hay NO identificadas
        else:
            print("No Identificados No Asignados Definitivamente")

    # Prints
    print("Número de filas en DataFrame PAS:", df_pas.shape[0])
    print("Número de filas en DataFrame Mitigacion:", df_mitigacion.shape[0])
    print("Número de filas en DataFrame Plan Seguimiento:", df_plan_seguimiento.shape[0])
    print("Número de filas en DataFrame Forma Cumplimiento:", df_forma_cumplimiento.shape[0])
    print("Número de filas en DataFrame Compromiso Voluntario:", df_compromiso_voluntario.shape[0])
    print("Número de filas en DataFrame Condiciones y Exigencias:", df_condiciones_exigencias.shape[0])
    print("Número de filas en DataFrame Contingencias Emergencias:", df_contingencias_emergencias.shape[0])
    print("Número de filas en DataFrame No Identificados:", df_no_identificados.shape[0])
    
    # Reordenar: 
    print("ok para reordenar 1")
    df_pas = df_pas[['numero_tabla', 'fase', 'parte_asociada', 'calificacion', 'condiciones', 'pronunciamiento', 'ice']]
    df_mitigacion = df_mitigacion[['numero_tabla', 'fase', 'tipo_medida', 'impacto', 'mitigacion', 'objetivo_descripcion_justificacion', 'lugar_forma_oportunidad', 'medio_verificacion', 'indicador_cumplimiento', 'ice']]     
    df_plan_seguimiento = df_plan_seguimiento[["numero_tabla", "componente_materia", "fase", "impacto", "mitigacion", "plazo_implementacion", "indicador_exito", "ubicacion", "parametros_a_medir", "limites_permitidos", "duracion_monitoreo", "frecuencia_monitoreo", "variable_ambiental", "duracion_frecuencia_medicion", "metodo_procedimiento_medicion", "plazo_frecuencia_informe", "destinatario"]]
    df_forma_cumplimiento = df_forma_cumplimiento[['numero_tabla', 'componente_materia', 'normas', 'otros_legal', 'fase', 'parte_asociada', 'lugar_forma_oportunidad', 'forma_cumplimiento', 'indicador_cumplimiento', 'control_seguimiento', 'ice']] 
    df_compromiso_voluntario = df_compromiso_voluntario[['numero_tabla', 'nombre_compromiso', 'impacto', 'fase', 'objetivo_descripcion_justificacion', 'lugar_forma_oportunidad', 'indicador_cumplimiento', 'control_seguimiento', 'ice']] 
    df_condiciones_exigencias = df_condiciones_exigencias[['numero_tabla', 'nombre_compromiso', 'impacto', 'fase', 'objetivo_descripcion_justificacion', 'lugar_forma_oportunidad', 'indicador_cumplimiento', 'control_seguimiento', 'ice']]
    df_contingencias_emergencias = df_contingencias_emergencias[['numero_tabla', 'riesgo_contingencia', 'fase', 'parte_asociada', 'acciones', 'control_seguimiento', 'ice', 'sma']] 
    
    # Asumiendo que ya tienes los DataFrames definidos
    df_pas = df_pas.loc[:, ~(df_pas == "").all()]
    df_plan_seguimiento = df_plan_seguimiento.loc[:, ~(df_plan_seguimiento == "").all()]
    df_mitigacion = df_mitigacion.loc[:, ~(df_mitigacion == "").all()]
    df_forma_cumplimiento = df_forma_cumplimiento.loc[:, ~(df_forma_cumplimiento == "").all()]
    df_compromiso_voluntario = df_compromiso_voluntario.loc[:, ~(df_compromiso_voluntario == "").all()]
    df_condiciones_exigencias = df_condiciones_exigencias.loc[:, ~(df_condiciones_exigencias == "").all()]
    df_contingencias_emergencias = df_contingencias_emergencias.loc[:, ~(df_contingencias_emergencias == "").all()]

    dfs_dict = {
        "pas": df_pas,
        "mitigacion": df_mitigacion,
        "plan_seguimiento": df_plan_seguimiento,
        "forma_cumplimiento": df_forma_cumplimiento,
        "compromisos_voluntarios": df_compromiso_voluntario,
        "condiciones_exigencias": df_condiciones_exigencias,
        "contingencias_emergencias": df_contingencias_emergencias,
        "no_identificados": df_no_identificados
    }
    return dfs_dict


def to_json(dfs_dict):
    # Set DFs
    df_pas = dfs_dict["pas"]
    df_mitigacion = dfs_dict["mitigacion"]
    df_plan_seguimiento = dfs_dict["plan_seguimiento"]
    df_forma_cumplimiento = dfs_dict["forma_cumplimiento"]
    df_compromiso_voluntario = dfs_dict["compromisos_voluntarios"]
    df_condiciones_exigencias = dfs_dict["condiciones_exigencias"]
    df_contingencias_emergencias = dfs_dict["contingencias_emergencias"]
    df_no_identificados = dfs_dict["no_identificados"]
    
    # Convierte dataframes a JSON
    df_pas = df_pas.to_json(orient='records', force_ascii=False)
    df_mitigacion = df_mitigacion.to_json(orient='records', force_ascii=False)
    df_plan_seguimiento = df_plan_seguimiento.to_json(orient='records', force_ascii=False)
    df_forma_cumplimiento = df_forma_cumplimiento.to_json(orient='records', force_ascii=False)
    df_compromiso_voluntario = df_compromiso_voluntario.to_json(orient='records', force_ascii=False)
    df_condiciones_exigencias = df_condiciones_exigencias.to_json(orient='records', force_ascii=False)
    df_contingencias_emergencias = df_contingencias_emergencias.to_json(orient='records', force_ascii=False)
    df_no_identificados = df_no_identificados.to_json(orient='records', force_ascii=False)

    dfs_dict = {
        "pas": df_pas,
        "mitigacion": df_mitigacion, 
        "forma_cumplimiento": df_forma_cumplimiento,
        "plan_seguimiento": df_plan_seguimiento, 
        "compromisos_voluntarios": df_compromiso_voluntario,
        "condiciones_exigencias": df_condiciones_exigencias,
        "contingencias_emergencias": df_contingencias_emergencias,
        "no_identificados": df_no_identificados,
    }
    return dfs_dict

# Instrucciones

In [ ]:
eg_input_cv_ce = """
            9.2. Monitoreo hábitat de Guayacán.
            Impacto asociado: Afectación de hábitat de flora y vegetación.
            Fase del Proyecto a la que aplica: Operación de la faena.
            Objetivo, descripción y justificación:
            - Objetivo: Evaluar el estado del hábitat forestal del Guayacán.
            - Descripción: Se realizará un análisis del hábitat $!"()"!=forestal del Guayacán, mediante monitoreos trimestrales de parámetros físicos y biológicos en el área de influencia del Proyecto. 
            - Justificación: El monitoreo trimestral de parámetros físicos y biológicos permitirá definir indicadores multidisciplinarios sobre el estado del hábitat forestal. Estos indicadores permitirán informar posibles cambios en la composición, estructura y/o dinámica del ecosistema.
            Lugar, forma y oportunidad de implementación:
            - Lugar: Área de influencia de las obras del Proyecto, según lo señalado al respecto en la DIA, Anexo 2-8.
            - Forma: Se realizarán campañas trimestrales,dondeunprofesionalingenieroforestalyunprofesionalecólogo, realizarán un diagnóstico del hábitat forestal. Para esto, se realizará una visita a terreno, donde se caracterizará, la calidad del suelo, las condiciones climáticas, el estado de la formación de bosque y presencia, en relación a riqueza y abundancia, de especies singulares.
            - Oportunidad: Durante toda la fase de operación del Proyecto.
            Indicador que acredite su cumplimiento:
            - Entrega de Informe de diagnóstico del ábitat forestal del Guayacán a Superintendencia del MEDIO AMBIENTE y Corporación Nacional Forestal de la Región de Valparaíso.
            Forma de control y seguimiento:
            - Campaña trimestral de seguimiento de parámetros y Elaboración de Informe de Diagnóstico del hábitat forestal del Guayacán. 
            Referencia al ICE para mayores detalles: ICE, numeral 11.1.2.
            
            9.3. Aplicación de Supresor de Polvos en Camino, conforme a la ley 19.300
            Fase del Proyecto a la que aplica    
            Construcción y operación.  
            Para validar las firmas de este documento usted debe ingresar a la siguiente url
             https://validador.sea.gob.cl/validar/2160382331
            Objetivo, descripción justificación   
            -Objetivo: Reducir el consumo de agua fresca por abatimiento de las emisiones 
            de material particulado en caminos de tierra.  
            -Descripción: Consiste en la aplicación de producto biodegradable como supresor 
            de polvo, del tipo Bovesoil o Sea Tac, o similar.  
            -Justificación: La necesidad de reducir el consumo de agua fresca, en 
            consideración de la escasez hídrica en el territorio involucrado. En efecto, El uso 
            de supresor de polvo permitirá reducir la frecuencia de riego. 
            Lugar, forma y  oportunidad de implementación 
            Desde el acceso de sector Peñablanca hasta instalaciones de espesador 
            del Proyecto.  El supresor se aplicará con camiones  Página 74   %  aljibes, de conformidada las 
            especificaciones del proveedor. La medida se implementará durante la construcción y operación 
            del Proyectooo, en consideración de la frecuencia que indique el proveedor, que 
            en el caso del Bovesoil es!de quince días, mientras que Sea Tac de 6 meses. 
            Indicador acredite que su cumplimiento   
            Registros de aplicación de supresor de polvo  
            Forma de control y seguimiento   
            La implementación de este Compromiso estará a cargo del supervisor de las obras en 
            terreno, durante la construcción, y del jefe dePlanta, durante la operacion. Conforme al Decreto Supremo N°400 de 2008. 
            Referencia al ICE para mayores detalles: 
            para mayor detalle ver tabla 7.1.12 del ICE del proyecto
            """

eg_output_cv_ce = """
            [
                {
                    "numero_tabla": "9.2",
                    "fase": "Operación de la faena",
                    "objetivo_descripcion_justificacion": "Objetivo: Evaluar el estado del hábitat forestal del Guayacán. Descripción: Se realizará un análisis del hábitat forestal del Guayacán, mediante monitoreos trimestrales de parámetros físicos y biológicos en el área de influencia del Proyecto. Justificación: El monitoreo trimestral de parámetros físicos y biológicos permitirá definir indicadores multidisciplinarios sobre el estado del hábitat forestal. Estos indicadores permitirán informar posibles cambios en la composición, estructura y/o dinámica del ecosistema.",
                    "lugar_forma_oportunidad": "Lugar: Área de influencia de las obras del Proyecto, según lo señalado al respecto en la DIA, Anexo 2-8. Forma: Se realizarán campañas trimestrales, donde un profesional ingeniero forestal y un profesional ecólogo, realizarán un diagnóstico del hábitat forestal. Para esto, se realizará una visita a terreno, donde se caracterizará, la calidad del suelo, las condiciones climáticas, el estado de la formación de bosque y presencia, en relación a riqueza y abundancia, de especies singulares. Oportunidad: Durante toda la fase de operación del Proyecto.",
                    "indicador_cumplimiento": "Entrega de Informe de diagnóstico del hábitat forestal del Guayacán a Superintendencia del Medio Ambiente y Corporación Nacional Forestal de la Región de Valparaíso.",
                    "control_seguimiento": "Campaña trimestral de seguimiento de parámetros y Elaboración de Informe de Diagnóstico del hábitat forestal del Guayacán. "
                },
                {
                    "numero_tabla": "9.3",
                    "fase": "Construcción y operación",
                    "objetivo_descripcion_justificacion": "Objetivo: Reducir el consumo de agua fresca por abatimiento de las emisiones de material particulado en caminos de tierra.  Descripción: Consiste en la aplicación de producto biodegradable como supresor de polvo, del tipo Bovesoil o Sea Tac, o similar.  Justificación: La necesidad de reducir el consumo de agua fresca, en consideración de la escasez hídrica en el territorio involucrado. En efecto, El uso de supresor de polvo permitirá reducir la frecuencia de riego",
                    "lugar_forma_oportunidad": "Desde el acceso de sector Peñablanca hasta instalaciones de espesador del Proyecto. El supresor se aplicará con camiones aljibes, de conformidad a las especificaciones del proveedor. La medida se implementará durante la construcción y operación del Proyecto, en consideración de la frecuencia que indique el proveedor, que en el caso del Bovesoil es de quince días, mientras que Sea Tac de 6 meses.",
                    "indicador_cumplimiento": "Registros de aplicación de supresor de polvo",
                    "control_seguimiento": "La implementación de este Compromiso estará a cargo del supervisor de las obras en terreno, durante la construcción, y del jefe de Planta, durante la operación. Conforme al Decreto Supremo N°400 de 2008."
                },
            ]
            """

sys_instr_cv_ce = """
             El texto se dividirá siempre en tablas que contendrán diversas variables. 
             
            ## Objetivo:
            Tu tarea es registrar tal como están en el texto original ciertos elementos de cada tabla (que pueden estar expresados de maneras ligeramente diferentes):
            - Extraer el "numero_tabla" el cuál estará al comenzar cada, en el formato especificado. 
            - Fases del proyecto a las que aplica o en las que se dará cumplimiento: registrar en "fase"
            - Objetivo, Descripción y Justificación: registrar todos sus elementos en "objetivo_descripcion_justificacion"
            - Lugar, Forma y Oportunidad de cumplimiento: registrar todos sus elementos en "lugar_forma_oportunidad"
            - Indicador que acredita su cumplimiento: registrar en "indicador_cumplimiento"
            - Forma de control y seguimiento: registrar en "control_seguimiento"
            Debes entregar absolutamente todo el texto completo asociado a cada campo buscado, sin excepción, por más extenso que sea.

            Debes asegurar que todas las variables tengan textos legibles:
            - Espacios entre palabras correctamente asignados.
            - Corregir errores ortográficos.
            - Eliminar textos extraños como "@$#%!opw¿".

            - Recuerda que la instrucción más relevante es: si el objetivo es entregar el texto completo entonces debes entregar absolutamente todo el texto completo asociado a dicho campo buscado, sin excepción. Si ves anexos o NO estas seguro de si dicho texto forma parte o no del campo, incluyelos igualmente. 
            - El texto "Para validar las firmas de este documento usted debe ingresar a la siguiente url https://validador.sea.gob.cl/validar/2162850911" o similares está al final de cada página y no debe considerarse. 
            - Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
            - NO confundas el "numero de tabla" con el número de referencia del ICE o otros capítulos referenciados en la tabla. Si no estás seguro de haberlo identificado, guardalo cómo "no identificado".
            """

system_instructions_1 = {
        "pas": f"""
            ## Contexto:
            Te proporcionaré un texto sobre los permisos ambientales sectoriales (PAS) de una Resolución de Calificación Ambiental (RCA) emitida por el Servicio de Evaluación Ambiental de Chile para la evaluación de un proyecto. 
            Dicho texto se dividirá siempre en tablas que contendrán las mismas variables: el nombre del permiso involucrado; las fases del proyecto a las que aplica o en las que se dará cumplimiento; la parte, obra, acción, emisión, residuo o sustancias a las que aplica; las condiciones o exigencias específicas para su otorgamiento, y el pronunciamiento del órgano competente. En algunos casos, también se incluye una variable de referencia al ICE para mayores detalles y una variable de calificación de la parte u obra. 
            
            ## Objetivo:
            Tu tarea es registrar tal como están en el texto original ciertos elementos de cada tabla (que pueden estar expresados de maneras ligeramente diferentes):
            - Extraer el "numero_tabla", en el formato especificado. 
            - Nombre del permiso involucrado: se regista en "pas" el número de artículo del reglamento del SEIA, RSEIA o SEA correspondiente.
            - Fases del proyecto a las que aplica o en las que se dará cumplimiento: registrar en la variable "fase" 
            - En algunos casos, también se incluye una variable de Parte, obra, acción, emisión, residuo o sustancias a las que aplica: registrar en la variable "parte_asociada".   
            - Condiciones o exigencias específicas para su otorgamiento: registrar en la variable "condiciones". 
             Debes entregar absolutamente todo el texto completo asociado a cada campo buscado, sin excepción, por más extenso que sea.  
            
            Debes asegurar que todas las variables tengan textos legibles:
            - Espacios entre palabras correctamente asignados.
            - Corregir errores ortográficos.
            - Eliminar textos extraños como "@$#%!opw¿".

            - Recuerda que la instrucción más relevante es: si el objetivo es entregar el texto completo entonces debes entregar absolutamente todo el texto completo asociado a dicho campo buscado, sin excepción. Si ves anexos o NO estas seguro de si dicho texto forma parte o no del campo, incluyelos igualmente. 
            - El texto "Para validar las firmas de este documento usted debe ingresar a la siguiente url https://validador.sea.gob.cl/validar/2162850911" o similares está al final de cada página y no debe considerarse. 
            - Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
            - NO confundas el "numero de tabla" con el número de referencia del ICE o otros capítulos referenciados en la tabla. Si no estás seguro de haberlo identificado, guardalo cómo "no identificado".
            """, 
    "eg_input_pas": """
            9.1.4. Permiso para la construcción, reparación, modificacióny ampliación de cualquier planta de tratamiento de basuras y desperdicios de cualquier clase o para la instalación de todo lugar destinado a la acumulación, selección, industrialización, comercio o disposición final de basuras y desperdicios de cualquier clase, del artículo 140 del Reglamento del SEIA.
            Fase del proyecto a la cual corresponde
            Construcción y operación.
            Parte, obra o acción a la que aplica
            Almacenamiento de residuos asimilables a domésticossss: 1) Patio dealmacenamiento de residuos sólidos, 
            2) Punto limpio sala de residuos sólidos (RSNP-1), 
            3) Punto limpio sala de residuos sólidos (RSNP-2), 
            4) Punto limpio sala de residuos sólidos (RSNP-3), 
            5) Punto limpio sala de residuos sólidos (RSNP-4) y 
            6) Punto limpio sala de residuos sólidos (RSNP-5). 
            Los contenidos técnicos y formales para acreditar los requisitos de cumplimiento se presentan en el Anexo V-8 de la Adenda complementaria.
            Para mayor detalle del permiso ambiental sectorial, ver Anexo N°10.9 “Permiso Ambiental Sectorial del Artículo N°140” y Anexo N°10.10 “Permiso Ambiental Sectorial del Artículo N°140”, ambos del EIA;  numeral 10.8, Anexo I.1 “Consolidado de Obras Formato Digital (SHP y KMZ)”
            Condiciones o exigencias específicas para su otorgamiento
            Se requiere de la capacitación inmediata del personal en manejo de residuos y basuras
            Pronunciamiento del órgano competente
            A través del Ord. N°14.256/2021 de fecha 30 de dici$%"(embre de 2021, la SEREMI de Salud de la Región de Atacamasepronunció conforme a los antecedentes.
            Referencia al ICE para mayores detalles
            Tabla 11.1.4 de referencia del numeral ICE.
            
            9.1.5. Permiso para todo sitio destinado al almacenamiento de residuos peligrosos, del artículo 142 del Reglamento del/(01 SEIA.
            Fase del proyecto a lnasjka cual corresponde
            Todas las fases del proyecto
            Parte, obra o acción a la que aplica
            Planta ADR: - 1 Bodega (RSP-2).
            Condiciones o exigencias específicas para su otorgamiento
            No hay.
            Pronunciamiento del órgano competente
            A través del Ord. N°14.256/2021 de fecha 30 de diciembre de 2021, la SEREMI de Salud de la Región de ATACAMA se pronunció conforme a los antecedentes.
            Referencia al ICE para mayores detalles
            Tabla 11.1.5 del ICE.
            """, 
    "eg_output_pas": """
            [
                {
                    "numero_tabla": "9.1.4",
                    "pas": "140",
                    "fase": "Construcción y operación",
                    "parte_asociada": "Almacenamiento de residuos asimilables a domésticos: 1) Patio de almacenamiento de residuos sólidos, 2) Punto limpio sala de residuos sólidos (RSNP-1), 3) Punto limpio sala de residuos sólidos (RSNP-2), 4) Punto limpio sala de residuos sólidos (RSNP-3), 5) Punto limpio sala de residuos sólidos (RSNP-4) y 6) Punto limpio sala de residuos sólidos (RSNP-5). Los contenidos técnicos y formales para acreditar los requisitos de cumplimiento se presentan en el Anexo V-8 de la Adenda complementaria. Para mayor detalle del permiso ambiental sectorial, ver Anexo N°10.9 “Permiso Ambiental Sectorial del Artículo N°140” y Anexo N°10.10 “Permiso Ambiental Sectorial del Artículo N°140”, ambos del EIA;  numeral 10.8, Anexo I.1 “Consolidado de Obras Formato Digital (SHP y KMZ)”.",
                    "condiciones": "Se requiere de la capacitación inmediata del personal en manejo de residuos y basuras"
                },
                {
                    "numero_tabla": "9.1.5",
                    "pas": "142",
                    "fase": "Todas las fases del proyecto",
                    "parte_asociada": "Planta ADR: - 1 Bodega (RSP-2).",
                    "condiciones": "No hay."
                }
            ]
        """, 
    "forma_cumplimiento": f"""
            ## Contexto:
            Te proporcionaré un fragmento de una Resolución de Calificación Ambiental (RCA) emitida por el Servicio de Evaluación Ambiental de Chile, que describe la forma en que un proyecto de inversión debe cumplir con las normativas ambientales.
            El texto se dividirá siempre en tablas que contendrán diversas variables. 
            
            ## Objetivo:
            Tu tarea es registrar tal como están en el texto original ciertos elementos de cada tabla (que pueden estar expresados de maneras ligeramente diferentes):
            - Extraer el "numero_tabla", en el formato especificado. 
            - Componente/Materia: Se debe registrar la variable "componente_materia" según el componente ambiental protegido, que puede estar en el título de la tabla también 
            - Las fuentes (leyes, reglamentos, decretos, circulares), que aparecen en las "normas", en "otros cuerpos legales" mencionados en cada tabla, y que también pueden aparecer en el título de la tabla como D.S. (Decreto Supremo), se deben registrar en la variable "normas" tal como aparezcan en el texto original
              - Si se presentan fuentes en múltiples partes de la tabla, se debe señalar en que parte aparece dicha tabla, por ejemplo: "Título: D.S. 204 del año 2008 Superintendencia de Salud. Normas: Ley 19.300 emitida por el Ministerio del Medio Ambiente. Otros cuerpos legales: No hay otros cuerpos legales relevantes". 
            - Fases del proyecto a las que aplica o en las que se dará cumplimiento: registrar en "fase"
            - Parte, obra, acción, emisión, residuo o sustancias a las que aplica: registrar en "parte_asociada"
            - Forma de cumplimiento: registrar en "forma_cumplimiento"
            - Indicador que acredita su cumplimiento: registrar en "indicador_cumplimiento"
            - Forma de control y seguimiento: registrar en "control_seguimiento"
            Debes entregar absolutamente todo el texto completo asociado a cada campo buscado, sin excepción, por más extenso que sea.  
            
            Debes asegurar que todas las variables tengan textos legibles:
            - Espacios entre palabras correctamente asignados.
            - Corregir errores ortográficos.
            - Eliminar textos extraños como "@$#%!opw¿".
            - Recuerda que la instrucción más relevante es: si el objetivo es entregar el texto completo entonces debes entregar absolutamente todo el texto completo asociado a dicho campo buscado, sin excepción. Si ves anexos o NO estas seguro de si dicho texto forma parte o no del campo, incluyelos igualmente. 
            - El texto "Para validar las firmas de este documento usted debe ingresar a la siguiente url https://validador.sea.gob.cl/validar/2162850911" o similares está al final de cada página y no debe considerarse. 
            - Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
            - NO confundas el "numero de tabla" con el número de referencia del ICE o otros capítulos referenciados en la tabla. Si no estás seguro de haberlo identificado, guardalo cómo "no identificado".
            """, 
    "eg_input_forma_cumplimiento": """
            10.1. Componente/materia: Explosivos.
            Norma: Decreto Supremo Nº 400 de 1977, y Ley N°17.798, Sobre Control de Armas, 06 de diciembre de 1977, Ministerio de Defensa Nacional.
            Otros cuerpos legales: Decreto Supremo Nº 83 de 2007, Aprueba Reglamento Complementario de la Ley Nº 17.798, Sobre Control de Armas y Elementos Similares Nacional.
            Fase del proyecto a la que aplica o en la que se dará cumplimiento: Durante las fases de la Construcción y operación.
            Parte, obra, acción, emisión, residuo o sustancias a la que aplica: Durante lasfases de construcción y de operación del proyecto, se requerirá del almacenamiento, utilización y transporte de explosivos para ser utilizados en las tronadoras necesarias para habilitar plataformas y los rajos, fracturando el material de sobrecarga y el mineral.
            Forma de cumplimiento: Los explosivos serán adquiridos, transportados, almacenadosymanipulados por empresascontratistasseqsmklañ que cumplan con las disposiciones requeridas por la Ley y sus reglamentos. El Titular se asegurará, previamente al contrato de estas empresas, que cuenten con todas las autorizaciones necesarias para estas labores.
            Indicador que acredita su cumplimiento: Certificados y autorizaciones vigentes otorgadaspor las autoridades competentes.
            Forma de control y seguimiento: Registro de las empresas abastecedoras de explosivos.
            Referencia al ICE para mayores detalles: Tabla 10.2.14 del numeral ICE del Proyecto.
            
            10.2. Emisiones de Vehículos
            Norma: D.S. N°279/1983. Reglamento para el Control de la Emisión de Contaminantes de Vehículos Motorizados de Combustión Interna.
            Otros cuerpos legales: D.S. N° 55/1994 del sadbj%$"Ministerio de Transportes y Telecomunicaciones, D.S. N° 54/1994 del Ministerio de Transportes y Telecomunicaciones, D.S. N° 211/1991 del Ministerio de Transportes y Telecomunicaciones.
            Componente/Materia: Control de Emisiones de Contaminantes de Vehículos Motorizados.
            Fase del Proyecto: Faaase de Construcción y Cierre.
            Forma de cumplimiento: Los vehículos motorizados livianos, así como camiones y maquinarias, contarán con sus revisiones técnicaSSSS$$$ anuales al día. Aquellos vehículos que no cumplan con este requisito no podrán ser admitidos en las diferentes actividades del Proyecto.
            Indicador que acredita su cumplimiento: Certificado y registro DE REVvisión Técnica y de Emisión de Gases al día. Registro de mantenciones de maquinaria. Registro mensual de vehículos que ingresan al Área del Proyecto.
            Control y seguimiento: Los registros señalados en los indicadores de cumplimiento se encontrarán disponibles para su revisión cuando la autoridad los solicite para su fiscalización.
            """, 
    "eg_output_forma_cumplimiento": """
             [
                {
                    "numero_tabla": "10.1",
                    "componente_materia": "Explosivos.", 
                    "normas": "Norma: Decreto Supremo Nº 400 de 1977, y Ley N°17.798, Sobre Control de Armas, 06 de diciembre de 1977, Ministerio de Defensa Nacional. Otros Cuerpos Legales: Decreto Supremo Nº 83 de 2007, Aprueba Reglamento Complementario de la Ley Nº 17.798, Sobre Control de Armas y Elementos Similares Nacional.",
                    "fase": "Durante las fases de la Construcción y operación", 
                    "parte_asociada": "Durante las fases de construcción y de operación del proyecto, se requerirá del almacenamiento, utilización y transporte de explosivos para ser utilizados en las tronadoras necesarias para habilitar plataformas y los rajos, fracturando el material de sobrecarga y el mineral.",
                    "forma_cumplimiento": "Los explosivos serán adquiridos, transportados, almacenados y manipulados por empresas contratistas que cumplan con las disposiciones requeridas por la Ley y sus reglamentos. El Titular se asegurará, previamente al contrato de estas empresas, que cuenten con todas las autorizaciones necesarias para estas labores.",
                    "indicador_cumplimiento": "Certificados y autorizaciones vigentes otorgadas por las autoridades competentes.",
                    "control_seguimiento": "Registro de las empresas abastecedoras de explosivos."
                },
                {
                    "numero_tabla": "10.2",
                    "componente_materia": "Control de Emisiones de Contaminantes de Vehículos Motorizados.", 
                    "normas": "Norma: D.S. N°279/1983. Reglamento para el Control de la Emisión de Contaminantes de Vehículos Motorizados de Combustión Interna. Otros Cuerpos Legales: D.S. N° 55/1994 del Ministerio de Transportes y Telecomunicaciones, D.S. N° 54/1994 del Ministerio de Transportes y Telecomunicaciones, D.S. N° 211/1991 del Ministerio de Transportes y Telecomunicaciones.",
                    "fase": "Fase de Construcción y Cierre.", 
                    "parte_asociada": "", 
                    "forma_cumplimiento": "Los vehículos motorizados livianos, así como camiones y maquinarias, contarán con sus revisiones técnicas anuales al día. Aquellos vehículos que no cumplan con este requisito no podrán ser admitidos en las diferentes actividades del Proyecto.",
                    "indicador_cumplimiento": "Certificado y registro de Revisión Técnica y de Emisión de Gases al día. Registro de mantenciones de maquinaria. Registro mensual de vehículos que ingresan al Área del Proyecto.",
                    "control_seguimiento": "Los registros señalados en los indicadores de cumplimiento se encontrarán disponibles para su revisión cuando la autoridad los solicite para su fiscalización."
                }    
            ]
          """, 
    "compromisos_voluntarios": f"""
            ## Contexto:
            Te proporcionaré el texto de los compromisos ambientales voluntarios (CAV) suscritos por una empresa en una Resolución de Calificación Ambiental emitida por el Servicio de Evaluación Ambiental de Chile para la evaluación de su proyecto. 
            {sys_instr_cv_ce}
            """, 
    "eg_input_compromisos_voluntarios": f"""{eg_input_cv_ce}""", 
    "eg_output_compromisos_voluntarios": f"""{eg_output_cv_ce}""", 
    "condiciones_exigencias": f"""
            ## Contexto:
            Te proporcionaré el texto de las condiciones y exigencias establecidas en una Resolución de Calificación Ambiental emitida por el Servicio de Evaluación Ambiental de Chile para la evaluación de un proyecto.
            {sys_instr_cv_ce}
            Si recibes texto debes entregar un resultado en formato tabla, que NO quede vacía
            """, 
    "eg_input_condiciones_exigencias": f"""{eg_input_cv_ce}""", 
    "eg_output_condiciones_exigencias": f"""{eg_output_cv_ce}""", 
    "mitigacion": f"""
            ## Contexto:
            Te proporcionaré el texto de las medidas de mitigación establecidas en una Resolución de Calificación Ambiental emitida por el Servicio de Evaluación Ambiental de Chile para la evaluación de un proyecto.
            {sys_instr_cv_ce}
            """, 
    "eg_input_mitigacion": f"""{eg_input_cv_ce}""", 
    "eg_output_mitigacion": f"""{eg_output_cv_ce}""", 
    "plan_seguimiento": f"""
            ## Contexto:
            Te proporcionaré el texto de las medidas del Plan de Seguimiento de una Resolución de Calificación Ambiental (RCA) emitida por el Servicio de Evaluación Ambiental de Chile para distintos proyectos. 
            El texto se dividirá siempre en tablas que contendrán diversas variables. 

            ## Objetivo:
            Tu tarea es registrar tal como están en el texto original ciertos elementos de cada tabla (que pueden estar expresados de maneras ligeramente diferentes y NO siempre estarán presentes):
            - Extraer el "numero_tabla", en el formato especificado. 
            - Fases del proyecto a las que aplica o en las que se dará cumplimiento: registrar en "fase"
            - Componente o Materia ambiental: registrar en "componente_materia"
            - Impacto asociado y/o Variables ambientales que serán objeto de muestreo, medición, análisis y/o control: registar en "impacto"
            - Medidas asociadas: registrar en "mitigacion"
            - Indicador de Éxito o de efectividad de la medida: registrar en "indicador_cumplimiento"
            - Parámetros a medir/parámetros que serán utilizados para caracterizar el estado y evolución de las variables ambientales: registrar en "control_seguimiento"
            - Los límites permitidos o comprometidos considerados en la evaluación: registrar en "condiciones"

            Registrar en la variable "lugar_forma_oportunidad" distintos atributos del texto entregado:
            - Ubicación de los puntos, zonas o sitios de muestreo, medición, análisis y/o control: registrar en "lugar"
            - Método o procedimiento de muestreo, medición, análisis y/o control para cada parámetro: registrar en "forma"
            - Plazo de implementación de la medida: registrar en "oportunidad"
            - Duración y frecuencia de las actividades de muestreo, medición, análisis y/o control para cada parámetro: registrar en "oportunidad"
            - Plazo y frecuencia de entrega de informes: registrar en "oportunidad"                 


            Debes entregar absolutamente todo el texto completo asociado a cada campo buscado, sin excepción, por más extenso que sea.

            Debes asegurar que todas las variables tengan textos legibles:
            - Espacios entre palabras correctamente asignados.
            - Corregir errores ortográficos.
            - Eliminar textos extraños como "@$#%!opw¿".

            - Recuerda que la instrucción más relevante es: si el objetivo es entregar el texto completo entonces debes entregar absolutamente todo el texto completo asociado a dicho campo buscado, sin excepción. Si ves anexos o NO estas seguro de si dicho texto forma parte o no del campo, incluyelos igualmente. 
            - El texto "Para validar las firmas de este documento usted debe ingresar a la siguiente url https://validador.sea.gob.cl/validar/2162850911" o similares está al final de cada página y no debe considerarse. 
            - Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
            - NO confundas el "numero de tabla" con el número de referencia del ICE o otros capítulos referenciados en la tabla. Si no estás seguro de haberlo identificado, guardalo cómo "no identificado".
            """, 
    "eg_input_plan_seguimiento": """
            8.1. Variable ambiental: Fauna Silvestre
            Impacto asociado CO FAU-001: Afectación de la población de Liolaemus rosenmanni
            (Lagartija de Rosenmanni), Liolaemus patriciaiturrae (Lagarto de
            Patricia Iturra) y Liolaemus isabelae (Lagartija de Isabel), debido a la
            intervención de superficies en que se identifican ambientes con la
            presencia de estas especies de reptiles.
            Medidas asociadas Medida 1: Plan de rescate y relocalización de reptiles.
            Medida 2: Perturbación controlada de reptiles en sectores de habilitación
            de obras lineales
            Componente ambiental objeto de seguimiento
            Animales silvestres
            Ubicación de los puntos/zonas de medición y control
            En la tabla siguiente se presentan las coordenadas de los sectores donde
            serán trasladados los ejemplares de fauna de baja y media movilidad,
            correspondiente a los sectores de Quebrada El Toro y Quebrada
            Codoceo.
            Sectores Superficie (ha) Este (m) Norte (m)
            Quebrada Codoceo 27 472.867 7.025.534
            Quebrada El Toro 4,8 478.573 7.018.886
            Quebrada del Toro: El sitio se encuentra ubicado al inicio de la Quebrada
            del Toro, con una superficie aproximada de 27 ha donde se relocalizarán
            los individuos de Liolaemus rosenmanni y Liolaemus patriciaiturrae. Se
            encuentra a una distancia de 850 m de las áreas de captura y presenta el
            ambiente áreas con escasa vegetación así como áreas desprovistas de
            vegetación.
            Quebrada Codoceo: El sitio abarca una superficie aproximada de 4,8 ha
            donde se relocalizarán los individuos de Liolaemus rosenmanni,
            Liolaemus isabelae y Liolaemus patriciaiturrae. Se encuentra a una
            distancia de 1.650 m de las áreas de captura y presenta el ambiente de
            matorral.
            Para las obras lineales, específicamente sector de camino de acceso se
            procederá con la ejecución de la medida de perturbación controlada, en
            el sector del camino de acceso conforme a las siguientes coordenadas.
            Este (m) Norte (m)
            Garita 478.405 7.017.318
            Punto medio 472.123 7.019.958
            Campamento 468.329 7.025.532
            Parámetros a monitorear Los parámetros que se utilizarán para realizar el seguimiento de los
            individuos relocalizados serán los siguientes:
            - Marcaje.
            - Mediciones de abundancia poblacional.
            - Densidad.
            - Distribución espacial.
            Para cuantificar datos poblacionales de las especies que sirvan como
            indicadores de éxito, dentro de los 45 días siguientes a la liberación, se
            evaluará, por ejemplo, a las hembras recapturadas preñadas en el área
            donde fueron relocalizadas. Posterior a este período, se realizará un
            seguimiento anual, durante la fase de construcción (un año) y el primer
            año de operación.
            Para el seguimiento de la perturbación controlada se analizará la riqueza
            del ensamble, la abundancia de las especies y el grado de desplazamiento
            de los animales perturbados.
            Límites permitidos o comprometidos
            No aplica.
            Duración y frecuencia de la medición
            Duración:
            Respecto de la Medida 1:
            - Se efectuarán 4 campañas de monitoreo a los 1 o 2 días, 7, 14 y 21 días
            posteriores a la relocalización, durante 4 jornadas de trabajo por cada
            campaña, siendo una jornada considerada entre las 08:00 y las 18:00
            horas.
            Respecto de la Medida 2:
            - El Plan de seguimiento se llevará a cabo en el hábitat receptor por un
            periodo total de un año, realizando una campaña en primavera y otra en
            verano. Para ello se revisitarán ocularmente los transectos
            preestablecidos durante el micro-ruteo asociado a la perturbación.
            Frecuencia:
            Respecto de la Medida 1:
            - Periodo de Seguimiento un año, que tendrá la fase de construcción con
            una campaña en verano y otra en primavera.
            Respecto de la Medida 2:
            - El Plan de seguimiento se llevará a cabo en el hábitat receptor por un
            periodo total de un año, realizando una campaña en primavera y otra en
            verano. Para ello se revisitarán ocularmente los transectos
            preestablecidos durante el micro-ruteo asociado a la perturbación.
            Método o procedimiento de medición de cada parámetro
            Se efectuarán transectos de ancho fijo con búsqueda dirigida
            (levantamiento de piedras, matorrales, entre otros). realizando, en la
            medida de lo posible, avistamiento directo (binoculares) y/o indirectos
            (fecas y madrigueras, entre otras evidencias), siendo innecesaria la
            metodología empleada durante el rescate, evitando así una perturbación
            mayor de las especies.
            Organismo destinatario de informes
            Superintendencia del Medio Ambiente, a través de su página web.
            Referencia al ICE para mayores detalles
            Tabla 9.1 del ICE.""", 
    "eg_output_plan_seguimiento": """
            [
            {
            "numero_tabla": "8.1",
            "componente_materia": "Fauna Silvestre",
            "fase": "",
            "impacto": "CO FAU-001: Afectación de la población de Liolaemus rosenmanni (Lagartija de Rosenmanni), Liolaemus patriciaiturrae (Lagarto de Patricia Iturra) y Liolaemus isabelae (Lagartija de Isabel), debido a la intervención de superficies en que se identifican ambientes con la presencia de estas especies de reptiles.", 
            "mitigacion": "Medida 1: Plan de rescate y relocalización de reptiles. Medida 2: Perturbación controlada de reptiles en sectores de habilitación de obras lineales", 
            "indicador_cumplimiento": "Marcaje, Mediciones de abundancia poblacional, Densidad, Distribución espacial. Cuantificación de datos poblacionales dentro de los 45 días posteriores a la liberación, evaluación de hembras recapturadas preñadas, y seguimiento anual durante la fase de construcción y el primer año de operación.",
            "lugar_forma_oportunidad": ""Lugar: En la tabla siguiente se presentan las coordenadas de los sectores donde serán trasladados los ejemplares de fauna de baja y media movilidad, correspondiente a los sectores de Quebrada El Toro y Quebrada Codoceo. Sectores Superficie (ha) Este (m) Norte (m) Quebrada Codoceo 27 472.867 7.025.534 Quebrada El Toro 4,8 478.573 7.018.886 Quebrada del Toro: El sitio se encuentra ubicado al inicio de la Quebrada del Toro, con una superficie aproximada de 27 ha donde se relocalizarán los individuos de Liolaemus rosenmanni y Liolaemus patriciaiturrae. Se encuentra a una distancia de 850 m de las áreas de captura y presenta el ambiente áreas con escasa vegetación así como áreas desprovistas de vegetación. Quebrada Codoceo: El sitio abarca una superficie aproximada de 4,8 ha donde se relocalizarán los individuos de Liolaemus rosenmanni, Liolaemus isabelae y Liolaemus patriciaiturrae. Se encuentra a una distancia de 1.650 m de las áreas de captura y presenta el ambiente de matorral. Para las obras lineales, específicamente sector de camino de acceso se procederá con la ejecución de la medida de perturbación controlada, en el sector del camino de acceso conforme a las siguientes coordenadas. Este (m) Norte (m) Garita 478.405 7.017.318 Punto medio 472.123 7.019.958 Campamento 468.329 7.025.532 . Forma: Se efectuarán transectos de ancho fijo con búsqueda dirigida (levantamiento de piedras, matorrales, entre otros). realizando, en la medida de lo posible, avistamiento directo (binoculares) y/o indirectos (fecas y madrigueras, entre otras evidencias), siendo innecesaria la metodología empleada durante el rescate, evitando así una perturbación mayor de las especies. Oportunidad: Duración: Respecto de la Medida 1: - Se efectuarán 4 campañas de monitoreo a los 1 o 2 días, 7, 14 y 21 días posteriores a la relocalización, durante 4 jornadas de trabajo por cada campaña, siendo una jornada considerada entre las 08:00 y las 18:00 horas. Respecto de la Medida 2: - El Plan de seguimiento se llevará a cabo en el hábitat receptor por un periodo total de un año, realizando una campaña en primavera y otra en verano. Para ello se revisitarán ocularmente los transectos preestablecidos durante el micro-ruteo asociado a la perturbación. Frecuencia: Respecto de la Medida 1: - Periodo de Seguimiento un año, que tendrá la fase de construcción con una campaña en verano y otra en primavera. Respecto de la Medida 2: - El Plan de seguimiento se llevará a cabo en el hábitat receptor por un periodo total de un año, realizando una campaña en primavera y otra en verano. Para ello se revisitarán ocularmente los transectos preestablecidos durante el micro-ruteo asociado a la perturbación. ", 
            "control_seguimiento": ""Los parámetros que se utilizarán para realizar el seguimiento de los individuos relocalizados serán los siguientes: - Marcaje. - Mediciones de abundancia poblacional. - Densidad. - Distribución espacial. Para cuantificar datos poblacionales de las especies que sirvan como indicadores de éxito, dentro de los 45 días siguientes a la liberación, se evaluará, por ejemplo, a las hembras recapturadas preñadas en el área donde fueron relocalizadas. Posterior a este período, se realizará un seguimiento anual, durante la fase de construcción (un año) y el primer año de operación. Para el seguimiento de la perturbación controlada se analizará la riqueza del ensamble, la abundancia de las especies y el grado de desplazamiento de los animales perturbados.",
            "condiciones": "No aplica"
            }
            ]
    """, 
    "contingencias_emergencias": f"""
            ## Contexto:
            Te proporcionaré el texto de las medidas relevantes del Plan de Prevención de Contingencias y del Plan de Emergencias de una Resolución de Calificación Ambiental (RCA) emitida por el Servicio de Evaluación Ambiental de Chile para distintos proyectos. 
            El texto se dividirá siempre en tablas que contendrán diversas variables. 

            ## Objetivo:
            Tu tarea es registrar tal como están en el texto original ciertos elementos de cada tabla (que pueden estar expresados de maneras ligeramente diferentes):
            - Extraer el "numero_tabla", en el formato especificado. 
            - Riego o contingencia: registrar en la variable "riesgo_contingencia", en algunos casos puede ser el título de la tabla
            - Fases del proyecto a las que aplica o en las que se dará cumplimiento: registrar en "fase"
            - Emplazamiento, parte, obra, acción, emisión, residuo o sustancias a las que aplica: registrar en "parte_asociada" 
            - Aacciones o medidas a implementar para prevenir la contingencia: si se presenta esta variable debes registrarla en "acciones_contingencia"
            - Acciones o medidas a implementar para controlar la emergencia: si se presenta esta variable debes registrarla en "acciones_contingencia"
            - En algunos casos sólo la tabla sólo señala acciones o medidas a implementar, sin aclarar si es para una contingencia o una emergencia, en dichos casos deberás determinar por el título y por el resto de la tabla si registrar en acciones_contingencia" o en "acciones_emergencia"
            - Forma de control y seguimiento: registrar en "control_seguimiento"
            Debes entregar absolutamente todo el texto completo asociado a cada campo buscado, sin excepción, por más extenso que sea.

            Debes asegurar que todas las variables tengan textos legibles:
            - Espacios entre palabras correctamente asignados.
            - Corregir errores ortográficos.
            - Eliminar textos extraños como "@$#%!opw¿".

            - Recuerda que la instrucción más relevante es: si el objetivo es entregar el texto completo entonces debes entregar absolutamente todo el texto completo asociado a dicho campo buscado, sin excepción. Si ves anexos o NO estas seguro de si dicho texto forma parte o no del campo, incluyelos igualmente. 
            - El texto "Para validar las firmas de este documento usted debe ingresar a la siguiente url https://validador.sea.gob.cl/validar/2162850911" o similares está al final de cada página y no debe considerarse. 
            - Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
            - NO confundas el "numero de tabla" con el número de referencia del ICE o otros capítulos referenciados en la tabla. Si no estás seguro de haberlo identificado, guardalo cómo "no identificado". 
            """, 
    "eg_input_contingencias_emergencias": """
            Tabla 10.1. Riesgo o contingencia: Incendio y/o Explosiones.
            Fase del proyecto a la que aplica: Construcción, Operación y Cierre.
            Emplazamiento, parte, obra o acción asociada: Todas las obras del proyecto.
            Acciones o medidas a implementar para prevenir la contingencia:
            - Control y concientización de la población fumadora mediante el establecimiento de una zona para fumadores.
            - Capacitar y entrenar al personal en técnicas de prevención y control de incendios, dejando registro mediante lista de asistencia.
            - Mantención de la operatividad y funcionalidad de la brigada de emergencias.
            - Sistema de extinción de incendio portátil.
            - Existencia de camiones aljibes tipo bombas.
            - Red de Sistema de agua contra incendio (red húmeda).
            Forma de control y seguimiento:
            - Establecimiento de zonas para las personas fumadoras.
            - Se dejará registro de los contenidos tratados y listas de asistencias firmadas por los trabajadores que reciben las capacitaciones.
            - Revisión y mantención periódica del sistema de extinción de incendios de los equipos mineros y maquinarias e instalaciones industriales.
            Acciones o medidas a implementar para controlar la emergencia:
            - Declarado el incendio, se activa la alarma sonora de incendio.
            - Se da aviso a camiones aljibes para quedar en estado de emergencia de incendio.
            - Jefe de brigada o líder de equipo lidera las primeras maniobras junto a los brigadistas para activación de red húmeda.
            - Luego se trabaja en conjunto brigada de rescate y bomberos de la comuna en el combate de incendio declarado.
            - Controlada la emergencia se realiza levantamiento de hallazgos y se inicia el periodo de investigación.
            Oportunidad y vías de comunicación a la SMA de la activación del Plan de Emergencia:
            Ante la ocurrencia de un incendio se considera el aviso inmediato a entidades externas al proyecto, siguiendo el protocolo establecido en el decreto N°400 del 2006 y en la ley N°19.300."

            Tabla 15.2. Riesgo o contingencia: Derrames accidentales de sustancias químicas no clasificadas como peligrosas.
            Fase del proyecto a la que aplica: Operación.
            Emplazamiento, parte, obra o acción asociada: Instalaciones de mantenimiento.
            Acciones o medidas a implementar para prevenir la contingencia:
            - Identificación y etiquetado adecuado de las sustancias químicas almacenadas.
            - Revisión periódica de los envases para detectar posibles fugas.
            - Establecimiento de protocolos de limpieza inmediata en caso de derrame.
            Forma de control y seguimiento:
            - Registro de las revisiones y mantenimiento de los envases.
            Acciones o medidas a implementar para controlar la emergencia:
            - Uso de materiales absorbentes no específicos para contener el derrame.
            - Disposición de los residuos en contenedores no especificados para sustancias no peligrosas.
            Oportunidad y vías de comunicación a la SMA de la activación del Plan de Emergencia:
            Aviso inmediato a la supervisión interna y registro de la incidencia en el libro de eventos del proyecto.
            """, 
    "eg_output_contingencias_emergencias": """
            [
                {
                    "numero_tabla": "10.1",
                    "riesgo_contingencia": "Incendio y/o Explosiones",
                    "fase": "Construcción, Operación y Cierre",
                    "parte_asociada": "Todas las obras del proyecto",
                    "acciones_contingencia": "- Control y concientización de la población fumadora mediante el establecimiento de una zona para fumadores. - Capacitar y entrenar al personal en técnicas de prevención y control de incendios, dejando registro mediante lista de asistencia. - Mantención de la operatividad y funcionalidad de la brigada de emergencias. - Sistema de extinción de incendio portátil. - Existencia de camiones aljibes tipo bombas. - Red de Sistema de agua contra incendio (red húmeda).",
                    "acciones_emergencia": "- Declarado el incendio, se activa la alarma sonora de incendio. - Se da aviso a camiones aljibes para quedar en estado de emergencia de incendio. - Jefe de brigada o líder de equipo lidera las primeras maniobras junto a los brigadistas para activación de red húmeda. - Luego se trabaja en conjunto brigada de rescate y bomberos de la comuna en el combate de incendio declarado. - Controlada la emergencia se realiza levantamiento de hallazgos y se inicia el periodo de investigación.", 
                    "control_seguimiento": "- Establecimiento de zonas para las personas fumadoras. - Se dejará registro de los contenidos tratados y listas de asistencias firmadas por los trabajadores que reciben las capacitaciones. - Revisión y mantención periódica del sistema de extinción de incendios de los equipos mineros y maquinarias e instalaciones industriales."
                },
                {
                    "numero_tabla": "15.2",
                    "riesgo_contingencia": "Derrames accidentales de sustancias químicas no clasificadas como peligrosas.",
                    "fase": "operación",
                    "parte_asociada": "Instalaciones de mantenimiento",
                    "acciones_contingencia": "- Identificación y etiquetado adecuado de las sustancias químicas almacenadas. - Revisión periódica de los envases para detectar posibles fugas. - Establecimiento de protocolos de limpieza inmediata en caso de derrame.",
                    "acciones_emergencia": "- Uso de materiales absorbentes no específicos para contener el derrame. - Disposición de los residuos en contenedores no especificados para sustancias no peligrosas.",
                    "control_seguimiento": "- Registro de las revisiones y mantenimiento de los envases."
                }
            ]
             """,
    "deduplicacion": f"""

    ## Contexto:
    Te proporcionaré un archivo JSON que contiene un conjunto de filas, cada una describiendo obligaciones presentes en una Resolución de Calificación Ambiental (RCA) de un proyecto de inversión en Chile.

    ## Objetivo:
    Tu tarea es eliminar las filas duplicadas, es decir, las filas con el mismo valor en la variable "numero_tabla" o filas con distinto "numero_tabla" pero el mismo contenido. 
    - Si hay diferencias en los valores de estas filas duplicadas, deberás combinarlas para formar una fila completa y coherente. El JSON resultante deberá cumplir con los formatos especificados para cada variable, y cada fila deberá estar ordenada de acuerdo con la estructura de la clase `FINAL`. 
    - El resultado final siempre deberá tener una sola fila por cada "numero_tabla".
    - Debes preservar la máxima fidelidad al texto original. Si una de las filas duplicadas contiene parte del contenido y la otra tiene otra porción, debes combinarlas sin repetir el contenido.
    - Si una variable está vacía en una fila duplicada, utiliza el contenido disponible en la otra fila. Si ambas están vacías, mantén el campo vacío. 
    - Las variable "seccion" debe mantenerse exactamente como está en las filas originales.
    - Las filas que no tienen duplicados deben mantenerse inalteradas.
    Te proporcionaré algunos ejemplos de entradas y salidas esperadas. Úsalos como guía para procesar nuevas entradas.
    Debes entregar Sólo 1 Fila Siempre

        """, 
    "eg_input_deduplicacion": """
        [
          {
            "seccion": "forma_cumplimiento",
            "numero_tabla": "2.1",
            "pas": "",
            "normas": "decreto 23/2006"
            "fase": "construcción y O",
            "parte_asociada": "Todas las etapas del proyecto",
            "componente_materia": "Gestión de Residuos Sólidos",
            "acciones_contingencia": "",
            "acciones_contingencia": "",
            "riesgo_contingencia": "",
            "indicador_cumplimiento": "Resoluciones sanitarias correspondientes a la aprobación del proyecto",
            "lugar_forma_oportunidad": "zona E7",
            "control_seguimiento": "Registro con información sobre la cantidad mensual de residuos generados, y registro de retiro de residuos a través del Sistema REP en RECT.",
            "forma_cumplimiento": "Respecto a los residuos industriales no peligrosos, se realizará un registro en el RETC, de forma anual. Los residuos peligrosos serán almacenados en recipientes sellados y debidamente etiquetados y trasladados hasta la bodega de almacenamiento de residuos peligrosos que se habilitará en el área del Proyecto, el cual cumplirá con los requisitos normativos correspondientes.",
            "condiciones": "",
            "objetivo_descripcion_justificacion": ""
          },
          {
            "seccion": "forma_cumplimiento",
            "numero_tabla": "2.1",
            "pas": "",
            "normas": "Ley 19300"
            "fase": "construcción y Operación",
            "parte_asociada": "",
            "componente_materia": "Gestión de Residuos Sólidos",
            "acciones_contingencia": "",
            "acciones_contingencia": "",
            "riesgo_contingencia": "",
            "indicador_cumplimiento": "Resoluciones sanitarias correspondientes a la aprobación del proyecto y de funcionamiento del almacenamiento temporal de residuos peligrosos. También, se tendrá un registro del retiro de cada tipo de residuo por un gestor autorizado para el tratamiento de éstos. Comprobante del registro en el Sistema REP (RETC), y declaración exigida por el MMA.",
            "lugar_forma_oportunidad": "zona E7",
            "control_seguimiento": "registro de retiro de residuos a través del Sistema REP en RECT.",
            "forma_cumplimiento": "Los residuos peligrosos serán almacenados en recipientes sellados y debidamente etiquetados y trasladados hasta la bodega de almacenamiento de residuos peligrosos que se habilitará en el área del Proyecto, el cual cumplirá con los requisitos normativos correspondientes. El transporte se realizará por empresas de terceros autorizados hasta el sitio disposición final autorizado."
            "condiciones": "",
            "objetivo_descripcion_justificacion": ""
          
          }
        ]
        """, 
    "eg_output_deduplicacion": """ 
        {
          "seccion": "forma_cumplimiento",
          "numero_tabla": "2.1",
          "pas": "",
          "normas": "Ley 19300 y decreto 23/2006"
          "fase": "construcción y Operación",
          "parte_asociada": "Todas las etapas del proyecto",
          "componente_materia": "Gestión de Residuos Sólidos",
          "acciones_contingencia": "",
          "acciones_contingencia": "",
          "riesgo_contingencia": "",
          "indicador_cumplimiento": "Resoluciones sanitarias correspondientes a la aprobación del proyecto y de funcionamiento del almacenamiento temporal de residuos peligrosos. También, se tendrá un registro del retiro de cada tipo de residuo por un gestor autorizado para el tratamiento de éstos. Comprobante del registro en el Sistema REP (RETC), y declaración exigida por el MMA.",
          "lugar_forma_oportunidad": "zona E7",
          "control_seguimiento": "Registro con información sobre la cantidad mensual de residuos generados, y registro de retiro de residuos a través del Sistema REP en RECT.",
          "contenido_obligacion": "Respecto a los residuos industriales no peligrosos, se realizará un registro en el RETC, de forma anual. Los residuos peligrosos serán almacenados en recipientes sellados y debidamente etiquetados y trasladados hasta la bodega de almacenamiento de residuos peligrosos que se habilitará en el área del Proyecto, el cual cumplirá con los requisitos normativos correspondientes. El transporte se realizará por empresas de terceros autorizados hasta el sitio de disposición final autorizado."
          "condiciones": "",
          "objetivo_descripcion_justificacion": ""
        }
    """
}


### Formatos Pydantic y Regex

In [ ]:

regex = r"^(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])|(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])$|^$"
regex_pas = r"\d{1,3}"

class Seccion(str, Enum):
    pas = "pas"
    mitigacion = "mitigacion"
    plan_seguimiento = "plan_seguimiento"
    forma_cumplimiento = "forma_cumplimiento"
    compromisos_voluntarios = "compromisos_voluntarios"
    condiciones_exigencias = "condiciones_exigencias"
    contingencias_emergencias = "contingencias_emergencias"


# Diccionario de descripciones comunes mejoradas
descriptions = {
    "pas": f"Número de artículo del reglamento del SEIA, RSEIA o SEA correspondiente, que debe registrarse en la variable 'pas' en formato regex {regex_pas}.",
    "numero_tabla": f"Identificador único de la tabla, que debe extraerse en el formato regex {regex}",
    "parte_asociada": f"Emplazamiento, parte, obra, acción, emisión, residuo o sustancias a las que aplica la condición descrita en la tabla. Debe registrarse tal como aparece en el texto original.",
    "fase": f"Fases del proyecto a las que aplica o en las que se dará cumplimiento. Debe registrarse tal como está en el texto original.",
    "componente_materia": f"Texto original que describe el componente o materia ambiental tratado en la tabla. Debe reflejar fielmente el contenido del documento fuente.",
    "control_seguimiento": f"Métodos o procedimientos descritos para el control y seguimiento del cumplimiento de la obligación o condición. Debe basarse en la información original del documento.",
    "condiciones": f"Condiciones o exigencias específicas para el otorgamiento, registradas tal como están en el texto original. Si no existen condiciones, la variable 'sin_condiciones' debe marcarse como 'True'.",
    "forma_cumplimiento": f"forma de control y seguimiento, copiada directamente del texto original.",
    "acciones_contingencia": f"acciones o medidas a implementar prevenir la contingencia, copiada directamente del texto original.", 
    "acciones_emergencia": f"acciones o medidas a implementar para controlar la emergencia, copiada directamente del texto original.", 
    "riesgo_contingencia": f"riesgo o contingencia, copiado directamente del texto original.", 
    "normas":  f"normas, D.S (Decreto Supremo), y otros cuerpos legales, copiada directamente del texto original.", 
    "indicador_cumplimiento": f"Indicador de cumplimiento. copiada directamente del texto original.",
    "objetivo_descripcion_justificacion": f"objetivo, descripción y justificación, copiada directamente del texto original.",
    "lugar_forma_oportunidad": f"lugar, formas y oportunidad de implementación, copiada directamente del texto original.", 
    "impacto":  f"Impacto Asociado, copiada directamente del texto original.",
    "mitigacion": f"Medida o Medidas Asociadas, , copiada directamente del texto original.", 
    "lugar_forma_oportunidad_PS": f"LUGAR: Ubicación de los puntos, zonas o sitios de muestreo, medición, análisis y/o control. FORMA: Método o procedimiento de muestreo, medición, análisis y/o control para cada parámetro. OPORTUNIDAD: Plazo para la implementación de la medida especificada, registrar variables tal como está en el texto original.",  
    "indicador_exito": f"Indicador de éxito o de efectividad de la medida descrita, registrado tal como aparece en el texto original.",
    "parametros_a_medir": f"Parámetros a medir o que serán utilizados para caracterizar el estado y evolución de las variables ambientales, copiados fielmente del texto.",
    "limites_permitidos": f"Límites permitidos o comprometidos considerados en la evaluación, copiados tal cual del documento original.",
    "impacto_variable_ambiental": f"Impacto Asociado o Variables ambientales objeto de muestreo, medición, análisis y/o control, copiados tal cual del texto original.",
}

# Definir modelos Pydantic con descripciones usando el diccionario mejorado
class PAS(BaseModel):
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    pas: str = Field(description=descriptions["pas"])
    fase: str = Field(description=descriptions["fase"])
    parte_asociada: Optional[str] = Field(description=descriptions["parte_asociada"])
    condiciones: str = Field(description=descriptions["condiciones"])

class FC(BaseModel):
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    componente_materia: str = Field(description=descriptions["componente_materia"])
    normas: str = Field(description=descriptions["normas"])
    fase: str = Field(description=descriptions["fase"])
    parte_asociada: Optional[str] = Field(description=descriptions["parte_asociada"])
    forma_cumplimiento: str = Field(description=descriptions["forma_cumplimiento"])
    indicador_cumplimiento: str = Field(description=descriptions["indicador_cumplimiento"])
    control_seguimiento: str = Field(description=descriptions["control_seguimiento"])

class CV_CE(BaseModel):
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    fase: str = Field(description=descriptions["fase"])
    componente_materia: str = Field(description=descriptions["componente_materia"])
    objetivo_descripcion_justificacion: str = Field(description=descriptions["objetivo_descripcion_justificacion"])
    lugar_forma_oportunidad: str = Field(description=descriptions["lugar_forma_oportunidad"])
    indicador_cumplimiento: str = Field(description=descriptions["indicador_cumplimiento"])
    control_seguimiento: str = Field(description=descriptions["control_seguimiento"])

class CE(BaseModel):
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    riesgo_contingencia: str = Field(description=descriptions["riesgo_contingencia"])
    fase: str = Field(description=descriptions["fase"])
    parte_asociada: str = Field(description=descriptions["parte_asociada"])
    acciones_contingencia: str = Field(description=descriptions["acciones_contingencia"])
    acciones_emergencia: str = Field(description=descriptions["acciones_emergencia"])
    control_seguimiento: str = Field(description=descriptions["control_seguimiento"])
    
class PS(BaseModel):
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    componente_materia: str = Field(description=descriptions["componente_materia"])
    fase: str = Field(description=descriptions["fase"])
    mitigacion: str = Field(description=descriptions["mitigacion"])
    impacto: str = Field(description=descriptions["impacto_variable_ambiental"])
    indicador_cumplimiento: str = Field(description=descriptions["indicador_exito"])
    lugar_forma_oportunidad: str = Field(description=descriptions["lugar_forma_oportunidad_PS"])
    control_seguimiento: str = Field(description=descriptions["parametros_a_medir"])
    condiciones: str = Field(description=descriptions["limites_permitidos"])

class FINAL(BaseModel):
    seccion: Seccion 
    numero_tabla: str = Field(description=descriptions["numero_tabla"])
    pas: str = Field(description=descriptions["pas"])
    normas: str = Field(description=descriptions["normas"])
    fase: str = Field(description=descriptions["fase"])
    parte_asociada: str = Field(description=descriptions["parte_asociada"])
    componente_materia: str = Field(description=descriptions["componente_materia"])
    acciones_contingencia: str = Field(description=descriptions["acciones_contingencia"])
    acciones_emergencia: str = Field(description=descriptions["acciones_emergencia"])
    riesgo_contingencia: str = Field(description=descriptions["riesgo_contingencia"])
    indicador_cumplimiento: str = Field(description=descriptions["indicador_cumplimiento"])
    lugar_forma_oportunidad: str = Field(description=descriptions["lugar_forma_oportunidad"])
    control_seguimiento: str = Field(description=descriptions["control_seguimiento"])
    forma_cumplimiento: str = Field(description=descriptions["forma_cumplimiento"])
    condiciones: str = Field(description=descriptions["condiciones"])
    objetivo_descripcion_justificacion: str = Field(description=descriptions["objetivo_descripcion_justificacion"])
    mitigacion: str = Field(description=descriptions["mitigacion"])
    impacto: str = Field(description=descriptions["impacto"])
    
class RF_Final(BaseModel):
    steps: List[FINAL]

class RF_PAS(BaseModel):
    steps: List[PAS]

class RF_Forma_Cumplimiento(BaseModel):
    steps: List[FC]

class RF_CV_CE(BaseModel):
    steps: List[CV_CE]

class RF_Contingencias_Emergencias(BaseModel):
    steps: List[CE]

class RF_Plan_Seguimiento(BaseModel):
    steps: List[PS]

response_format = {
    "mitigacion": RF_CV_CE,
    "plan_seguimiento": RF_Plan_Seguimiento,
    "pas": RF_PAS,
    "forma_cumplimiento": RF_Forma_Cumplimiento,
    "compromisos_voluntarios": RF_CV_CE,
    "condiciones_exigencias": RF_CV_CE,
    "contingencias_emergencias": RF_Contingencias_Emergencias,
    "final": RF_Final
}


# Funciones Principales

###  Tokenización y Función Open AI

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def count_tokens(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return len(tokens)

def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, 
        chunk_overlap=500,
        length_function=count_tokens, 
        is_separator_regex=False
    )
    return text_splitter.split_text(documents)

In [ ]:
def calcular_costo_response(response):
    # Tarifas estimadas por modelo (en dólares por 1,000 tokens)
    tarifas = {
    'gpt-5-nano-2025-08-07': {
        'prompt': 0.00015,
        'completion': 0.00060
    },
    }

    modelo = response.model
    tokens_prompt = response.usage.prompt_tokens
    tokens_completion = response.usage.completion_tokens

    if modelo not in tarifas:
        raise ValueError(f"Modelo no encontrado en tabla de tarifas: {modelo}")

    tarifa_prompt = tarifas[modelo]['prompt']
    tarifa_completion = tarifas[modelo]['completion']

    costo_prompt = (tokens_prompt / 1000) * tarifa_prompt
    costo_completion = (tokens_completion / 1000) * tarifa_completion
    costo_total = costo_prompt + costo_completion

    return {
        'modelo': modelo,
        'tokens_prompt': tokens_prompt,
        'tokens_completion': tokens_completion,
        'costo_prompt': f"{costo_prompt:.6f} USD",
        'costo_completion': f"{costo_completion:.6f} USD",
        'costo_total': f"{costo_total:.6f} USD"
    }


def gpt_4o_mini(client, text, system_instructions_1, response_format_used, initial_messages,
                total_cost=0.0, total_input_tokens=0, total_output_tokens=0):

    # Armar la lista de mensajes incluyendo la instrucción del sistema y el mensaje del usuario
    messages = copy.deepcopy(initial_messages)
    messages.insert(0, {"role": "system", "content": system_instructions_1})
    messages.append({"role": "user", "content": text})

    # Solicitar la respuesta a la API
    completion = client.beta.chat.completions.parse(
        model="gpt-5-nano",
        response_format=response_format_used,
        messages=messages,
    )

    
    # Procesar la respuesta (se asume que el contenido es un JSON con la clave "steps")
    response_content = completion.choices[0].message.content
    response_dict = json.loads(response_content)
    df = pd.DataFrame(response_dict["steps"])

    tarifas = {
        'gpt-5-nano-2025-08-07': {
            'prompt': 0.00015,
            'completion': 0.00060
        },
    }

    modelo = completion.model
    tokens_prompt = completion.usage.prompt_tokens
    tokens_completion = completion.usage.completion_tokens

    if modelo not in tarifas:
        raise ValueError(f"Modelo no encontrado en tabla de tarifas: {modelo}")

    tarifa_prompt = tarifas[modelo]['prompt']
    tarifa_completion = tarifas[modelo]['completion']

    costo_prompt = (tokens_prompt / 1000) * tarifa_prompt
    costo_completion = (tokens_completion / 1000) * tarifa_completion
    costo_total = costo_prompt + costo_completion

    # Actualizar totales
    nuevo_total_cost = total_cost + costo_total
    nuevo_total_input_tokens = total_input_tokens + tokens_prompt
    nuevo_total_output_tokens = total_output_tokens + tokens_completion

    # Preparar diccionario con los totales actualizados
    totales_actualizados = {
        'total_cost': f"{nuevo_total_cost:.6f} USD",
        'total_input_tokens': nuevo_total_input_tokens,
        'total_output_tokens': nuevo_total_output_tokens,
        'detalle_uso_actual': {
            'modelo': modelo,
            'tokens_prompt': tokens_prompt,
            'tokens_completion': tokens_completion,
            'costo_prompt': f"{costo_prompt:.6f} USD",
            'costo_completion': f"{costo_completion:.6f} USD",
            'costo_total': f"{costo_total:.6f} USD"
        }
    }

    # Se retorna tanto el DataFrame con los pasos como los totales actualizados
    return df, totales_actualizados




### Loop Identificación de Obligaciones

In [ ]:
def loop_ai(sections_text, dfs_dict, total_input_tokens=0, total_output_tokens=0):
    tablas_identificadas = ("Para facilitar tu trabajo, también proporcionaré una tabla JSON "
                            "(la cual puede contener errores y puede No coincidir exactamente con el texto, sólo es una guía)")
    dfs = []
    # Lista de secciones a procesar
    sections_to_process = [section for section in sections_text.keys() if section in [
        'mitigacion', 'plan_seguimiento', 'pas', 'forma_cumplimiento', 
        'compromisos_voluntarios', 'contingencias_emergencias', 'condiciones_exigencias'
    ]]
    # Recorrer cada sección
    for section in sections_to_process:
        # Verificar el número de tokens de la sección
        if count_tokens(sections_text[section]) > 4500:
            # Dividir el texto de la sección en fragmentos
            section_chunks = split_text_into_chunks(sections_text[section])
        else:
            section_chunks = [sections_text[section]]

        # Procesar cada fragmento de la sección
        for i, chunk in enumerate(section_chunks, 1):
            text = f"- Texto Base: {chunk}\n\n {tablas_identificadas}:\n{dfs_dict[section]}"
            if dfs_dict[section] == '[]':
                text = chunk
                print(f"Chunk procesado sin tablas asociadas")
            
            # Usar la función openai
            try:
                initial_messages = [
                    {"role": "user", "content": system_instructions_1[f"eg_input_{section}"]},
                    {"role": "assistant", "content": system_instructions_1[f"eg_output_{section}"]},
                ]
                # Llamada a gpt_4o_mini sin preocuparnos por el costo
                df, totales_actualizados = gpt_4o_mini(
                    client, text, system_instructions_1[section],
                    response_format[section], initial_messages,
                    0.0, total_input_tokens, total_output_tokens  
                )
                df['seccion'] = section
                df['chunk'] = i
                print(f"Procesado chunk {i} de la sección {section}, de {count_tokens(chunk)} tokens")
                
                # Solo actualizamos contadores de tokens
                total_input_tokens = totales_actualizados.get('total_input_tokens', total_input_tokens)
                total_output_tokens = totales_actualizados.get('total_output_tokens', total_output_tokens)
            
            except Exception as e:
                print(f"Error {e} al procesar chunk {i}, intentando nuevamente")
                try:
                    initial_messages = [
                        {"role": "user", "content": system_instructions_1[f"eg_input_{section}"]},
                        {"role": "assistant", "content": system_instructions_1[f"eg_output_{section}"]},
                    ]
                    df, totales_actualizados = gpt_4o_mini(
                        client, text, system_instructions_1[section],
                        response_format[section], initial_messages,
                        0.0, total_input_tokens, total_output_tokens
                    )
                    df['seccion'] = section
                    df['chunk'] = i
                    print(f"Procesado chunk {i} de la sección {section}, de {count_tokens(chunk)} tokens")
                    
                    # Solo actualizamos contadores de tokens
                    total_input_tokens = totales_actualizados.get('total_input_tokens', total_input_tokens)
                    total_output_tokens = totales_actualizados.get('total_output_tokens', total_output_tokens)
                
                except Exception as e2:
                    print(f"Error {e2} nuevamente")
                    df = pd.DataFrame()
                    pass
            
            # Guardar el DataFrame en la lista
            dfs.append(df)

    # Concatenar todos los DataFrames generados
    df_final = pd.concat(dfs, ignore_index=True)
    
    # Retornar el DataFrame final y solo los tokens acumulados
    return df_final, total_input_tokens, total_output_tokens

### Identificar Tablas Faltantes

In [ ]:
def identificar_tabla_faltante(tabla_serie):

    resultado = tabla_serie.copy()
    n = len(resultado)
    usados = set(resultado)  # Para asegurarnos de no duplicar números ya asignados
    
    for i in range(n):
        try: 
            if "no.identificado" in resultado[i]:
                anterior = resultado[i-1] if i > 0 else None
                posterior = resultado[i+1] if i < n - 1 else None
                
                if anterior and posterior and "no.identificado" not in anterior and "no.identificado" not in posterior:
                    anterior_split = [int(part) for part in anterior.split('.')]
                    posterior_split = [int(part) for part in posterior.split('.')]
                    
                    # Caso donde anterior y posterior son iguales
                    if anterior == posterior:
                        asignado = anterior
                        resultado[i] = asignado
                        usados.add(asignado)
                        print(f"Asignando {resultado[i]} en la fila {i} entre {anterior} y {posterior}")
                    
                    # Caso de diferencia de un número entero
                    elif len(anterior_split) == len(posterior_split) and posterior_split[0] - anterior_split[0] == 2:
                        asignado = '.'.join(map(str, [anterior_split[0] + 1] + anterior_split[1:]))
                        resultado[i] = asignado
                        usados.add(asignado)
                        print(f"Asignando {resultado[i]} en la fila {i} entre {anterior} y {posterior}")
                    
                    # Caso de diferencia de un subnivel
                    elif anterior_split[:-1] == posterior_split[:-1] and len(anterior_split) == len(posterior_split):
                        if posterior_split[-1] - anterior_split[-1] == 2:
                            nuevo_valor = anterior_split[:-1] + [anterior_split[-1] + 1]
                            asignado = '.'.join(map(str, nuevo_valor))
                            resultado[i] = asignado
                            usados.add(asignado)
                            print(f"Asignando {resultado[i]} en la fila {i} entre {anterior} y {posterior}")
                        else:
                            print(f"No se puede asignar un valor adecuado en la fila {i} entre {anterior} y {posterior}")
                    else:
                        print(f"No se puede asignar un valor adecuado en la fila {i} entre {anterior} y {posterior}")
                else:
                    pass
                    #print(f"No hay suficiente información para asignar en la fila {i}")
        except Exception as e:
            print(f"Error al procesar la fila {i}: {str(e)}")
    return resultado

### Formatear

In [ ]:
def formatear(dfs, cell):
    dfs = dfs.copy()
    # Asegurar presencias de todas las claves en conceptos_dict
    required_columns = [
        "seccion",
        "chunk",
        "numero_tabla",
        "pas",
        "normas",
        "fase", 
        "parte_asociada",
        "componente_materia",
        "acciones_contingencia",
        "acciones_emergencia",
        "riesgo_contingencia",
        "indicador_cumplimiento",
        "lugar_forma_oportunidad",
        "control_seguimiento",
        "forma_cumplimiento", 
        "condiciones", 
        "objetivo_descripcion_justificacion",      
        "impacto",
        "mitigacion", 
    ]

    for column in required_columns:
        if column not in dfs.columns:
            dfs[column] = ""
    dfs = dfs[required_columns]
    dfs['chunk'] = dfs['chunk'].astype(str)
    dfs.fillna("", inplace=True)
    
    try:
        dfs['numero_tabla'] = identificar_tabla_faltante(dfs['numero_tabla'])
    except:
        print('error en identificación de tablas no identificadas')

    dfs = dfs.drop(columns=['chunk'])
    grouped = dfs.groupby('numero_tabla')
    df_final = pd.DataFrame()
    
    # Variables para acumular tokens en formatear
    total_input_formatear = 0
    total_output_formatear = 0
    
    regex = r"^(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])|(?:[1-9]\d?|[1-9])\.(?:[1-9]\d?|[1-9])$|^$"
    
    for numero_tabla, group in grouped:
        if re.match(regex, numero_tabla) and len(group) > 1 and not (any(group['seccion'].isin(["forma_cumplimiento", "mitigacion"]))):
            try:
                print(f"Procesando {len(group)} filas con numero_tabla {numero_tabla}")
                dfs_json = group.to_json(orient='records', force_ascii=False)
                initial_messages = [
                    {"role": "user", "content": system_instructions_1["eg_input_deduplicacion"]},
                    {"role": "assistant", "content": system_instructions_1["eg_output_deduplicacion"]},
                ]
                # Obtener correctamente los valores de retorno de gpt_4o_mini
                df_aux, totales_actualizados = gpt_4o_mini(
                    client, dfs_json, system_instructions_1["deduplicacion"], 
                    response_format["final"], initial_messages
                )
                # Acumular tokens
                total_input_formatear += totales_actualizados.get('total_input_tokens', 0)
                total_output_formatear += totales_actualizados.get('total_output_tokens', 0)
                
                df_final = pd.concat([df_final, df_aux], ignore_index=True)
    
            except Exception as e:
                print(f"error: {e}")
                try: 
                    print(f"Reintentando Proceso {len(group)} filas con numero_tabla {numero_tabla}")
                    dfs_json = group.to_json(orient='records', force_ascii=False)
                    initial_messages = [
                        {"role": "user", "content": system_instructions_1["eg_input_deduplicacion"]},
                        {"role": "assistant", "content": system_instructions_1["eg_output_deduplicacion"]},
                    ]
                    df_aux, totales_actualizados = gpt_4o_mini(
                        client, dfs_json, system_instructions_1["deduplicacion"], 
                        response_format["final"], initial_messages
                    )
                    # Acumular tokens
                    total_input_formatear += totales_actualizados.get('total_input_tokens', 0)
                    total_output_formatear += totales_actualizados.get('total_output_tokens', 0)
                    
                    df_final = pd.concat([df_final, df_aux], ignore_index=True)
    
                except Exception as e:
                    print(f"error nuevamente: {e}")
                    longest_row = group.loc[group.apply(lambda row: len(''.join(row.astype(str))), axis=1).idxmax()]
                    df_final = pd.concat([df_final, pd.DataFrame([longest_row])], ignore_index=True)
        
        else:
            print(f"Numero_tabla {numero_tabla} no tiene duplicados, no se procesa.")
            df_final = pd.concat([df_final, group], ignore_index=True)

    # Retornar solo tokens en lugar de costo
    return df_final, total_input_formatear, total_output_formatear

# Ejecución

### Listado SEIA

In [ ]:
# Ruta del archivo Excel
file_path = os.path.join(direccion_output, f"seia_{sector}.xlsx")

# Leer el archivo Excel usando pandas
df = pd.read_excel(file_path, engine='openpyxl')

# Abrir el archivo Excel con openpyxl para obtener los hipervínculos
workbook = openpyxl.load_workbook(file_path, data_only=True)
sheet = workbook.active

# Función para extraer hipervínculos de una celda
def extract_hyperlinks(sheet):
    hyperlinks = {}
    for row in sheet.iter_rows():
        for cell in row:
            if cell.hyperlink:
                hyperlinks[cell.coordinate] = cell.hyperlink.target
    return hyperlinks

# Obtener los hipervínculos del archivo
hyperlinks = extract_hyperlinks(sheet)

# Crear nuevas columnas "cell" y "hyperlink" en el DataFrame
df["cell"] = ""
df["hyperlink"] = ""

# Asignar las coordenadas de las celdas y los hipervínculos a las columnas correspondientes
for i, (cell, hyperlink) in enumerate(hyperlinks.items()):
    if i < len(df):
        df.at[i, "hyperlink"] = hyperlink


### Procesamiento Completo

In [ ]:
# Inicializamos el DataFrame para guardar los costos por celda
df_costs = pd.DataFrame(columns=['cell', 'total_cost', 'total_input_tokens', 'total_output_tokens'])
df_final = pd.DataFrame()

# Definimos las tarifas aquí para usarlas al final
tarifas = {
    'gpt-5-nano-2025-08-07': {
        'prompt': 0.00015,
        'completion': 0.00060
    },
}

for i, cell in df["id"][:].items():
    try:
        print(cell)
        file_name = os.path.join(data_path, f"{cell}.pdf")

        # OBTENER TEXTO DE CADA SECCIÓN
        dfx = pd.DataFrame()
        document_text = extract_pdf_text(file_name)
        if not document_text.strip() or re.fullmatch(r'\s+', document_text):
            file_name = os.path.join(data_path, f"{cell}_TXT.txt")
            print("ESCANEADO")
            with open(file_name, 'r') as file:
                document_text = file.read()

        sections_text = separador_secciones(document_text)
        print(sections_text.keys())

        # OBTENER TABLAS DE CADA SECCIÓN 
        no_identif = False
        dfs_dict_vacio = {
            "pas": '[]',
            "mitigacion": '[]',
            "plan_seguimiento": '[]', 
            "forma_cumplimiento": '[]',
            "compromisos_voluntarios": '[]',
            "condiciones_exigencias": '[]',
            "contingencias_emergencias": '[]'
        }

        try:
            dfx, all_tables, percentage_unidentified = extraer_tablas(file_name, conceptos_dict, cell)
            dfs_dict0 = separacion_tablas_secciones(dfx, list(sections_text.keys()))
            if "no_identificados" not in dfs_dict0:
                dfs_dict0["no_identificados"] = []
            dfs_dict = to_json(dfs_dict0)
        except Exception as e:
            print("Tablas no Identificadas:", e)
            dfs_dict = dfs_dict_vacio
            no_identif = True

        num_tokens = count_tokens(document_text)
        print(f'Número de tokens del archivo {cell}: {num_tokens}')

        # Inicializamos contadores de tokens para la celda actual
        total_input_tokens_cell = 0
        total_output_tokens_cell = 0

        # PROCESAMIENTO CON TABLAS (si se identificaron)
        if not no_identif:
            dfs_con_tablas, input_tokens_tab, output_tokens_tab = loop_ai(sections_text, dfs_dict)
            total_input_tokens_cell += input_tokens_tab
            total_output_tokens_cell += output_tokens_tab
            print("Procesado por IA con tablas")
            print(f"Total input tokens: {input_tokens_tab}")
            print(f"Total output tokens: {output_tokens_tab}")
        else:
            dfs_con_tablas = pd.DataFrame()

        # PROCESAMIENTO SIN TABLAS
        dfs_sin_tablas, input_tokens_sin, output_tokens_sin = loop_ai(sections_text, dfs_dict_vacio)
        total_input_tokens_cell += input_tokens_sin
        total_output_tokens_cell += output_tokens_sin
        print("Procesado por IA sin tablas")
        print(f"Total input tokens: {input_tokens_sin}")
        print(f"Total output tokens: {output_tokens_sin}")

        # Verificar que sean DataFrames
        if not isinstance(dfs_con_tablas, pd.DataFrame):
            dfs_con_tablas = pd.DataFrame(dfs_con_tablas)
        if not isinstance(dfs_sin_tablas, pd.DataFrame):
            dfs_sin_tablas = pd.DataFrame(dfs_sin_tablas)

        # Combinar resultados de IA
        if not no_identif:
            dfs_concatenado = df_con_tablas
        else:
            dfs_concatenado = dfs_sin_tablas

        # Se llama a formatear y se capturan los tokens
        dfs_concatenado, input_tokens_form, output_tokens_form = formatear(dfs_concatenado, cell)
        total_input_tokens_cell += input_tokens_form
        total_output_tokens_cell += output_tokens_form
        print("Deduplicación tablas completada")
        print(f"Tokens de formateo - Input: {input_tokens_form}, Output: {output_tokens_form}")
        dfs_concatenado["cell"] = cell

        # Calcular costo total al final usando los tokens acumulados
        costo_prompt = (total_input_tokens_cell / 1000) * tarifas['gpt-5-nano-2025-08-07']['prompt']
        costo_completion = (total_output_tokens_cell / 1000) * tarifas['gpt-5-nano-2025-08-07']['completion']
        total_cost_cell = costo_prompt + costo_completion

        print("Resumen para celda", cell)
        print(f"Costo total calculado: {total_cost_cell:.6f} USD")
        print("Total input tokens acumulados:", total_input_tokens_cell)
        print("Total output tokens acumulados:", total_output_tokens_cell)

        # Agregar fila a df_costs
        nueva_fila = pd.DataFrame({
            'cell': [cell],
            'total_cost': [total_cost_cell],
            'total_input_tokens': [total_input_tokens_cell],
            'total_output_tokens': [total_output_tokens_cell]
        })
        df_costs = pd.concat([df_costs, nueva_fila], ignore_index=True)

        # GUARDAR resultados finales
        df_final = pd.concat([df_final, dfs_concatenado], ignore_index=True)
        df_final.to_excel(os.path.join(direccion_output, f'df_subconsiderandos_5-mini.xlsx'), index=False)

    except Exception as e:
        print(f"Error procesando {cell}: {e}")

# Guardar df_costs al finalizar
df_costs.to_excel(os.path.join(direccion_output, 'df_costs_.xlsx'), index=False)
print(df_costs)

In [ ]:
df_costs.to_excel(os.path.join(direccion_output, 'df_costs_.xlsx'), index=False)
df_final.to_excel(os.path.join(direccion_output, f'df_subconsiderandos.xlsx'), index=False)